<a href="https://colab.research.google.com/github/Ayisharya/Ayisharya/blob/main/matformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install networkx
!pip3 install pandas
!pip3 install plotly
!pip install pymatgen==2023.7.11
!pip3 install alignn
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html
!pip3 install pytz==2021.1
!pip3 install scipy==1.7.1
!pip3 install torch==2.0.0 torchvision==0.15.1 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118
!pip3 install jarvis-tools==2022.9.16
!pip3 install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip3 install torch_geometric
!pip install gensim
!pip install periodictable
!pip install pandarallel==1.6.3
!pip install pytorch-ignite

!pip install pydantic==1.10.4

In [ ]:
import pydantic
print(f"Pydantic: {pydantic.__version__}")

Pydantic: 1.10.4


In [ ]:
!git clone https://github.com/YKQ98/Matformer.git


fatal: destination path 'Matformer' already exists and is not an empty directory.


In [ ]:
pwd

'/content'

In [ ]:
%cd Matformer/

/content/Matformer


In [ ]:
pwd

'/content/Matformer'

In [ ]:
from Matformer.matformer.models import pyg_att

Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.0+cu118


In [ ]:
import sys
sys.path.append("/content/ALIGNN")

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import math
import os
import yaml

from jarvis.db.figshare import data as jdata
from jarvis.core.atoms import Atoms
from jarvis.core.graphs import Graph
import dgl

from ignite.engine import Engine, Events
from ignite.metrics import MeanSquaredError

# Import Matformer components
from Matformer.matformer.models.pyg_att import Matformer, MatformerConfig

# -------------------------
# Data Preparation Functions
# -------------------------
def collate_batch(batch):
    """Custom collate function for Matformer input"""
    graphs = []
    line_graphs = []
    lattices = []
    labels = []
    for item in batch:
        graphs.append(item[0][0])
        line_graphs.append(item[0][1])
        lattices.append(item[0][2])
        labels.append(item[1])
    batched_graphs = dgl.batch(graphs)
    batched_line_graphs = dgl.batch(line_graphs)
    batched_lattices = torch.stack(lattices)
    labels = torch.stack(labels)
    return (batched_graphs, batched_line_graphs, batched_lattices), labels

def load_dataset(name="dft_3d", target="formation_energy_peratom", limit=None):
    """Load dataset from JARVIS"""
    d = jdata(name)
    data = []
    for i in d:
        if target in i and i[target] != "na" and not math.isnan(i[target]):
            data.append(i)
    if limit is not None:
        data = data[:limit]
    return pd.DataFrame(data)

def create_graphs(df, target):
    """Create DGL graphs from atomic structures"""
    data = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            atoms = Atoms.from_dict(row["atoms"])
            graph, line_graph = Graph.atom_dgl_multigraph(atoms)
            lattice = torch.tensor(atoms.lattice_mat, dtype=torch.float)
            label = torch.tensor([row[target]], dtype=torch.float)
            data.append(((graph, line_graph, lattice), label))
        except Exception as e:
            print("Skipping graph due to error:", e)
            continue
    return data

# -------------------------
# Model Configuration
# -------------------------
def load_matformer_config():
    """Load Matformer configuration similar to pyg_att.py implementation"""
    config = {
        "name": "matformer",
        "node_dim": 64,
        "edge_dim": 64,
        "output_dim": 1,
        "link": "identity",
        "n_heads": 4,
        "n_layers": 3,
        "dropout": 0.0,
        "attention_dropout": 0.0,
        "use_bn": False,
        "use_ffn": False,
        "use_ffn_output": True,
        "use_residual": True,
        "use_attention_output": True,
        "hidden_dim": 128,
        "graph_norm": True,
        "edge_embed": False,
        "use_edge_attr": False,
        "pos_embed": False,
        "use_global_pool": True,
        "global_pool": "mean",
        "ffn_act": "relu",
        "ffn_norm": "batch",
        "attn_type": "multihead",
        "attn_kernel": "softmax",
        "attn_norm": "layer",
        "attn_act": "relu",
        "attn_mask": False,
        "attn_dropout": 0.0,
        "attn_residual": True,
        "attn_bias": True,
        "attn_ffn": False,
        "attn_ffn_act": "relu",
        "attn_ffn_norm": "batch",
        "attn_ffn_dropout": 0.0,
        "attn_ffn_residual": True,
        "attn_ffn_bias": True,
    }
    return MatformerConfig(**config)

# -------------------------
# Main Training Setup
# -------------------------
# Configuration
dataset_name = "dft_3d"
target_property = "formation_energy_peratom"
batch_size = 32
epochs = 10
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and prepare data
print("Loading dataset...")
df = load_dataset(name=dataset_name, target=target_property, limit=1000)
print(f"Loaded {len(df)} samples.")

data = create_graphs(df, target_property)
print(f"Graphs created: {len(data)}")

# Split data
train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size + val_size]
test_data = data[train_size + val_size:]

# Create data loaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

# Initialize Matformer model
config = load_matformer_config()
model = Matformer(config).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

# -------------------------
# Training Functions
# -------------------------
def train_step(engine, batch):
    """Training step function"""
    model.train()
    optimizer.zero_grad()
    (g, lg, lat), labels = batch
    g, lg, lat, labels = g.to(device), lg.to(device), lat.to(device), labels.to(device)

    # Forward pass with Matformer model
    outputs = model(g, lg, lat).view(-1)
    loss = criterion(outputs, labels.view(-1))

    loss.backward()
    optimizer.step()
    return loss.item()

def val_step(engine, batch):
    """Validation step function"""
    model.eval()
    with torch.no_grad():
        (g, lg, lat), labels = batch
        g, lg, lat, labels = g.to(device), lg.to(device), lat.to(device), labels.to(device)
        outputs = model(g, lg, lat).view(-1)
        return outputs, labels.view(-1)

# Create engines
trainer = Engine(train_step)
evaluator = Engine(val_step)
MeanSquaredError().attach(evaluator, "mse")

# -------------------------
# Logging and Checkpointing
# -------------------------
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    """Log validation metrics after each epoch"""
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print(f"Validation Results - Epoch: {engine.state.epoch}  MSE: {metrics['mse']:.4f}")

os.makedirs("checkpoints", exist_ok=True)

@trainer.on(Events.EPOCH_COMPLETED)
def save_checkpoint(engine):
    """Save model checkpoint after each epoch"""
    epoch = engine.state.epoch
    checkpoint_path = f"checkpoints/matformer_model_{epoch}.pt"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'config': config,
    }, checkpoint_path)
    print(f"Saved checkpoint to {checkpoint_path}")

# -------------------------
# Run Training
# -------------------------
print("Starting training...")
trainer.run(train_loader, max_epochs=epochs)
print("Training completed!")

Loading dataset...
Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
Loaded 1000 samples.


100%|██████████| 1000/1000 [00:33<00:00, 29.53it/s]


Graphs created: 1000


ValidationError: 35 validation errors for MatformerConfig
attention_dropout
  extra fields not permitted (type=value_error.extra)
attn_act
  extra fields not permitted (type=value_error.extra)
attn_bias
  extra fields not permitted (type=value_error.extra)
attn_dropout
  extra fields not permitted (type=value_error.extra)
attn_ffn
  extra fields not permitted (type=value_error.extra)
attn_ffn_act
  extra fields not permitted (type=value_error.extra)
attn_ffn_bias
  extra fields not permitted (type=value_error.extra)
attn_ffn_dropout
  extra fields not permitted (type=value_error.extra)
attn_ffn_norm
  extra fields not permitted (type=value_error.extra)
attn_ffn_residual
  extra fields not permitted (type=value_error.extra)
attn_kernel
  extra fields not permitted (type=value_error.extra)
attn_mask
  extra fields not permitted (type=value_error.extra)
attn_norm
  extra fields not permitted (type=value_error.extra)
attn_residual
  extra fields not permitted (type=value_error.extra)
attn_

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class DGLMatformer(nn.Module):
    def __init__(self, config):
        super(DGLMatformer, self).__init__()
        self.node_dim = config["node_features"]
        self.edge_dim = config["edge_features"]
        self.output_dim = config["output_features"]

        # Simple linear layers to embed nodes and edges
        self.node_emb = nn.Linear(1, self.node_dim)
        self.edge_emb = nn.Linear(1, self.edge_dim)

        # GNN Layers (simplified Transformer-like blocks)
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=self.node_dim,
                nhead=config["node_layer_head"],
                dim_feedforward=self.node_dim*2,
                batch_first=True
            ) for _ in range(config["conv_layers"])
        ])

        # Output layer
        self.output_layer = nn.Sequential(
            nn.Linear(self.node_dim + 9, self.node_dim),  # 9 for flattened 3x3 lattice
            nn.ReLU(),
            nn.Linear(self.node_dim, self.output_dim)
        )

    def forward(self, inputs):
        g, lg, lattice = inputs  # lg is unused but passed for compatibility

        # Node and edge features
        h = self.node_emb(g.ndata['atom_features'].float())
        e = self.edge_emb(g.edata['edge_features'].float())

        # Assign to graph
        g.ndata['h'] = h
        g.edata['e'] = e

        # Aggregate node features with mean (can be replaced with attention or message passing)
        hg = dgl.mean_nodes(g, 'h')

        # Flatten lattice and concatenate
        flat_lat = lattice.view(lattice.size(0), -1)
        out = torch.cat([hg, flat_lat], dim=1)

        return self.output_layer(out)


In [ ]:
import sys
sys.path.append("/content/ALIGNN")

import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import math
import os

from jarvis.db.figshare import data as jdata
from jarvis.core.atoms import Atoms
from jarvis.core.graphs import Graph
from Matformer.matformer.models.pyg_att import MatformerConfig  # For config template
import dgl
import numpy as np

from ignite.engine import Engine, Events
from ignite.metrics import MeanSquaredError

# -------------------------
# DGLMatformer Model Definition
# -------------------------
import torch.nn as nn
import torch.nn.functional as F

class DGLMatformer(nn.Module):
    def __init__(self, config):
        super(DGLMatformer, self).__init__()
        self.node_dim = config["node_features"]
        self.edge_dim = config["edge_features"]
        self.output_dim = config["output_features"]

        self.node_emb = nn.Linear(1, self.node_dim)
        self.edge_emb = nn.Linear(1, self.edge_dim)

        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=self.node_dim,
                nhead=config["node_layer_head"],
                dim_feedforward=self.node_dim * 2,
                batch_first=True
            ) for _ in range(config["conv_layers"])
        ])

        self.output_layer = nn.Sequential(
            nn.Linear(self.node_dim + 9, self.node_dim),
            nn.ReLU(),
            nn.Linear(self.node_dim, self.output_dim)
        )

    def forward(self, inputs):
        g, lg, lattice = inputs

        h = self.node_emb(g.ndata['atom_features'].float())
        e = self.edge_emb(g.edata['edge_features'].float())

        g.ndata['h'] = h
        g.edata['e'] = e

        hg = dgl.mean_nodes(g, 'h')
        flat_lat = lattice.view(lattice.size(0), -1)
        out = torch.cat([hg, flat_lat], dim=1)

        return self.output_layer(out)

# -------------------------
# Data Preparation
# -------------------------
def collate_batch(batch):
    graphs, line_graphs, lattices, labels = [], [], [], []
    for item in batch:
        graphs.append(item[0][0])
        line_graphs.append(item[0][1])
        lattices.append(item[0][2])
        labels.append(item[1])
    return (
        dgl.batch(graphs),
        dgl.batch(line_graphs),
        torch.stack(lattices),
    ), torch.stack(labels)

def load_dataset(name="dft_3d", target="formation_energy_peratom", limit=None):
    d = jdata(name)
    data = []
    for i in d:
        if target in i and i[target] != "na" and not math.isnan(i[target]):
            data.append(i)
    if limit:
        data = data[:limit]
    return pd.DataFrame(data)

def create_graphs(df, target):
    data = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            atoms = Atoms.from_dict(row["atoms"])
            graph, line_graph = Graph.atom_dgl_multigraph(atoms)

            graph.ndata['atom_features'] = torch.tensor(atoms.atomic_numbers, dtype=torch.long).unsqueeze(-1)

            if 'pos' not in graph.ndata:
                graph.ndata['pos'] = torch.zeros(graph.num_nodes(), 3)

            if 'bond_dist' not in graph.edata:
                src, dst = graph.edges()
                src_pos = graph.ndata['pos'][src]
                dst_pos = graph.ndata['pos'][dst]
                dist = torch.norm(src_pos - dst_pos, dim=1)
                graph.edata['bond_dist'] = dist

            graph.edata['edge_features'] = graph.edata['bond_dist'].float().unsqueeze(-1)
            lattice = torch.tensor(atoms.lattice_mat, dtype=torch.float)
            label = torch.tensor([row[target]], dtype=torch.float)

            data.append(((graph, line_graph, lattice), label))
        except Exception as e:
            print("Skipping sample due to error:", e)
    return data

# -------------------------
# Main Training Setup
# -------------------------
dataset_name = "dft_3d"
target_property = "formation_energy_peratom"
batch_size = 32
epochs = 10
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Loading dataset...")
df = load_dataset(name=dataset_name, target=target_property, limit=1000)
print(f"Loaded {len(df)} samples.")

data = create_graphs(df, target_property)
print(f"Graphs created: {len(data)}")
if len(data) == 0:
    raise ValueError("No valid graphs created.")

train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size + val_size]
test_data = data[train_size + val_size:]

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

# -------------------------
# Model & Training Engine
# -------------------------
config_dict = {
    "name": "matformer",
    "node_features": 128,
    "edge_features": 128,
    "output_features": 1,
    "conv_layers": 5,
    "node_layer_head": 4,
}

model = DGLMatformer(config_dict).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

def train_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    (g, lg, lat), labels = batch
    g, lg, lat, labels = g.to(device), lg.to(device), lat.to(device), labels.to(device)

    outputs = model((g, lg, lat)).view(-1)
    loss = criterion(outputs, labels.view(-1))
    loss.backward()
    optimizer.step()
    return loss.item()

def val_step(engine, batch):
    model.eval()
    with torch.no_grad():
        (g, lg, lat), labels = batch
        g, lg, lat, labels = g.to(device), lg.to(device), lat.to(device), labels.to(device)
        outputs = model((g, lg, lat)).view(-1)
        return outputs, labels.view(-1)

trainer = Engine(train_step)
evaluator = Engine(val_step)
MeanSquaredError().attach(evaluator, "mse")

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    print(f"Validation - Epoch {engine.state.epoch} | MSE: {metrics['mse']:.4f}")

os.makedirs("checkpoints", exist_ok=True)

@trainer.on(Events.EPOCH_COMPLETED)
def save_checkpoint(engine):
    epoch = engine.state.epoch
    path = f"checkpoints/matformer_model_{epoch}.pt"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'config': config_dict,
    }, path)
    print(f"Saved checkpoint to {path}")

# -------------------------
# Train
# -------------------------
print("Starting training...")
trainer.run(train_loader, max_epochs=epochs)
print("Training completed.")

Loading dataset...
Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
Loaded 1000 samples.


100%|██████████| 1000/1000 [00:36<00:00, 27.31it/s]


Graphs created: 1000
Starting training...
Validation - Epoch 1 | MSE: 1.1486
Saved checkpoint to checkpoints/matformer_model_1.pt
Validation - Epoch 2 | MSE: 1.1309
Saved checkpoint to checkpoints/matformer_model_2.pt
Validation - Epoch 3 | MSE: 0.9725
Saved checkpoint to checkpoints/matformer_model_3.pt
Validation - Epoch 4 | MSE: 1.0108
Saved checkpoint to checkpoints/matformer_model_4.pt
Validation - Epoch 5 | MSE: 0.9432
Saved checkpoint to checkpoints/matformer_model_5.pt
Validation - Epoch 6 | MSE: 1.0081
Saved checkpoint to checkpoints/matformer_model_6.pt
Validation - Epoch 7 | MSE: 1.5548
Saved checkpoint to checkpoints/matformer_model_7.pt
Validation - Epoch 8 | MSE: 0.8910
Saved checkpoint to checkpoints/matformer_model_8.pt
Validation - Epoch 9 | MSE: 0.8671
Saved checkpoint to checkpoints/matformer_model_9.pt
Validation - Epoch 10 | MSE: 0.8624
Saved checkpoint to checkpoints/matformer_model_10.pt
Training completed.


In [ ]:
"""Implementation based on the template of ALIGNN."""

import imp
import random
from pathlib import Path
from typing import Optional

# from typing import Dict, List, Optional, Set, Tuple

import os
import torch
import numpy as np
import pandas as pd
from jarvis.core.atoms import Atoms
from matformer.graphs import PygGraph, PygStructureDataset
#
from jarvis.db.figshare import data as jdata
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
from jarvis.db.jsonutils import dumpjson

# from sklearn.pipeline import Pipeline
import pickle as pk

from sklearn.preprocessing import StandardScaler

# use pandas progress_apply
tqdm.pandas()


def load_dataset(
    name: str = "dft_3d",
    target=None,
    limit: Optional[int] = None,
    classification_threshold: Optional[float] = None,
):
    """Load jarvis data."""
    d = jdata(name)
    data = []
    for i in d:
        if i[target] != "na" and not math.isnan(i[target]):
            if classification_threshold is not None:
                if i[target] <= classification_threshold:
                    i[target] = 0
                elif i[target] > classification_threshold:
                    i[target] = 1
                else:
                    raise ValueError(
                        "Check classification data type.",
                        i[target],
                        type(i[target]),
                    )
            data.append(i)
    d = data
    if limit is not None:
        d = d[:limit]
    d = pd.DataFrame(d)
    return d


def mean_absolute_deviation(data, axis=None):
    """Get Mean absolute deviation."""
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)



def load_pyg_graphs(
    df: pd.DataFrame,
    name: str = "dft_3d",
    neighbor_strategy: str = "k-nearest",
    cutoff: float = 8,
    max_neighbors: int = 12,
    cachedir: Optional[Path] = None,
    use_canonize: bool = False,
    use_lattice: bool = False,
    use_angle: bool = False,
):
    """Construct crystal graphs.

    Load only atomic number node features
    and bond displacement vector edge features.

    Resulting graphs have scheme e.g.
    ```
    Graph(num_nodes=12, num_edges=156,
          ndata_schemes={'atom_features': Scheme(shape=(1,)}
          edata_schemes={'r': Scheme(shape=(3,)})
    ```
    """

    def atoms_to_graph(atoms):
        """Convert structure dict to DGLGraph."""
        structure = Atoms.from_dict(atoms)
        return PygGraph.atom_dgl_multigraph(
            structure,
            neighbor_strategy=neighbor_strategy,
            cutoff=cutoff,
            atom_features="atomic_number",
            max_neighbors=max_neighbors,
            compute_line_graph=False,
            use_canonize=use_canonize,
            use_lattice=use_lattice,
            use_angle=use_angle,
        )

    graphs = df["atoms"].progress_apply(atoms_to_graph).values

    return graphs


def get_id_train_val_test(
    total_size=1000,
    split_seed=123,
    train_ratio=None,
    val_ratio=0.1,
    test_ratio=0.1,
    n_train=None,
    n_test=None,
    n_val=None,
    keep_data_order=False,
):
    """Get train, val, test IDs."""
    if (
        train_ratio is None
        and val_ratio is not None
        and test_ratio is not None
    ):
        if train_ratio is None:
            assert val_ratio + test_ratio < 1
            train_ratio = 1 - val_ratio - test_ratio
            print("Using rest of the dataset except the test and val sets.")
        else:
            assert train_ratio + val_ratio + test_ratio <= 1
    # indices = list(range(total_size))
    if n_train is None:
        n_train = int(train_ratio * total_size)
    if n_test is None:
        n_test = int(test_ratio * total_size)
    if n_val is None:
        n_val = int(val_ratio * total_size)
    ids = list(np.arange(total_size))
    if not keep_data_order:
        random.seed(split_seed)
        random.shuffle(ids)
    if n_train + n_val + n_test > total_size:
        raise ValueError(
            "Check total number of samples.",
            n_train + n_val + n_test,
            ">",
            total_size,
        )

    id_train = ids[:n_train]
    id_val = ids[-(n_val + n_test) : -n_test]  # noqa:E203
    id_test = ids[-n_test:]
    return id_train, id_val, id_test


def get_torch_dataset(
    dataset=[],
    id_tag="jid",
    target="",
    neighbor_strategy="",
    atom_features="",
    use_canonize="",
    name="",
    line_graph="",
    cutoff=8.0,
    max_neighbors=12,
    classification=False,
    output_dir=".",
    tmp_name="dataset",
):
    """Get Torch Dataset."""
    df = pd.DataFrame(dataset)
    # print("df", df)
    vals = df[target].values
    if target == "shear modulus" or target == "bulk modulus":
        val_list = [vals[i].item() for i in range(len(vals))]
        vals = val_list
    print("data range", np.max(vals), np.min(vals))
    print("data mean and std", np.mean(vals), np.std(vals))
    f = open(os.path.join(output_dir, tmp_name + "_data_range"), "w")
    line = "Max=" + str(np.max(vals)) + "\n"
    f.write(line)
    line = "Min=" + str(np.min(vals)) + "\n"
    f.write(line)
    f.close()

    graphs = load_graphs(
        df,
        name=name,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
    )

    data = StructureDataset(
        df,
        graphs,
        target=target,
        atom_features=atom_features,
        line_graph=line_graph,
        id_tag=id_tag,
        classification=classification,
    )
    return data

def get_pyg_dataset(
    dataset=[],
    id_tag="jid",
    target="",
    neighbor_strategy="",
    atom_features="",
    use_canonize="",
    name="",
    line_graph="",
    cutoff=8.0,
    max_neighbors=12,
    classification=False,
    output_dir=".",
    tmp_name="dataset",
    use_lattice=False,
    use_angle=False,
    data_from='Jarvis',
    use_save=False,
    mean_train=None,
    std_train=None,
    now=False, # for test
):
    """Get pyg Dataset."""
    df = pd.DataFrame(dataset)
    # print("df", df)
    # neighbor_strategy = "pairwise-k-nearest"

    vals = df[target].values
    if target == "shear modulus" or target == "bulk modulus":
        val_list = [vals[i].item() for i in range(len(vals))]
        vals = val_list
    output_dir = "./saved_data/" + tmp_name + "test_graph_angle.pkl" # for fast test use
    print("data range", np.max(vals), np.min(vals))
    print(output_dir)
    if now:
        if not os.path.exists(output_dir):
            graphs = load_pyg_graphs(
                df,
                name=name,
                neighbor_strategy=neighbor_strategy,
                use_canonize=use_canonize,
                cutoff=cutoff,
                max_neighbors=max_neighbors,
                use_lattice=use_lattice,
                use_angle=use_angle,
            )
            with open(output_dir, 'wb') as pf:
                pk.dump(graphs, pf)
            print('save graphs to ', output_dir)
        else:
            print('loading graphs from ', output_dir)
            with open(output_dir, 'rb') as pf:
                graphs = pk.load(pf)
    else:
        print('graphs not saved')
        graphs = load_pyg_graphs(
            df,
            name=name,
            neighbor_strategy=neighbor_strategy,
            use_canonize=use_canonize,
            cutoff=cutoff,
            max_neighbors=max_neighbors,
            use_lattice=use_lattice,
            use_angle=use_angle,
        )
    if mean_train == None:
        mean_train = np.mean(vals)
        std_train = np.std(vals)
        data = PygStructureDataset(
            df,
            graphs,
            target=target,
            atom_features=atom_features,
            line_graph=line_graph,
            id_tag=id_tag,
            classification=classification,
            neighbor_strategy=neighbor_strategy,
            mean_train=mean_train,
            std_train=std_train,
        )
    else:
        data = PygStructureDataset(
            df,
            graphs,
            target=target,
            atom_features=atom_features,
            line_graph=line_graph,
            id_tag=id_tag,
            classification=classification,
            neighbor_strategy=neighbor_strategy,
            mean_train=mean_train,
            std_train=std_train,
        )
    return data, mean_train, std_train


def get_train_val_loaders(
    dataset: str = "dft_3d",
    dataset_array=[],
    target: str = "formation_energy_peratom",
    atom_features: str = "cgcnn",
    neighbor_strategy: str = "k-nearest",
    n_train=None,
    n_val=None,
    n_test=None,
    train_ratio=None,
    val_ratio=0.1,
    test_ratio=0.1,
    batch_size: int = 5,
    standardize: bool = False,
    line_graph: bool = True,
    split_seed: int = 123,
    workers: int = 0,
    pin_memory: bool = True,
    save_dataloader: bool = False,
    filename: str = "sample",
    id_tag: str = "jid",
    use_canonize: bool = False,
    cutoff: float = 8.0,
    max_neighbors: int = 12,
    classification_threshold: Optional[float] = None,
    target_multiplication_factor: Optional[float] = None,
    standard_scalar_and_pca=False,
    keep_data_order=False,
    output_features=1,
    output_dir=None,
    matrix_input=False,
    pyg_input=False,
    use_lattice=False,
    use_angle=False,
    use_save=True,
    mp_id_list=None,
):
    """Help function to set up JARVIS train and val dataloaders."""
    # data loading
    mean_train=None
    std_train=None
    assert (matrix_input and pyg_input) == False

    train_sample = filename + "_train.data"
    val_sample = filename + "_val.data"
    test_sample = filename + "_test.data"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if (
        os.path.exists(train_sample)
        and os.path.exists(val_sample)
        and os.path.exists(test_sample)
        and save_dataloader
    ):
        print("Loading from saved file...")
        print("Make sure all the DataLoader params are same.")
        print("This module is made for debugging only.")
        train_loader = torch.load(train_sample)
        val_loader = torch.load(val_sample)
        test_loader = torch.load(test_sample)
        if train_loader.pin_memory != pin_memory:
            train_loader.pin_memory = pin_memory
        if test_loader.pin_memory != pin_memory:
            test_loader.pin_memory = pin_memory
        if val_loader.pin_memory != pin_memory:
            val_loader.pin_memory = pin_memory
        if train_loader.num_workers != workers:
            train_loader.num_workers = workers
        if test_loader.num_workers != workers:
            test_loader.num_workers = workers
        if val_loader.num_workers != workers:
            val_loader.num_workers = workers
        print("train", len(train_loader.dataset))
        print("val", len(val_loader.dataset))
        print("test", len(test_loader.dataset))
        return (
            train_loader,
            val_loader,
            test_loader,
            train_loader.dataset.prepare_batch,
        )
    else:
        if not dataset_array:
            d = jdata(dataset)
        else:
            d = dataset_array
            # for ii, i in enumerate(pc_y):
            #    d[ii][target] = pc_y[ii].tolist()

        dat = []
        if classification_threshold is not None:
            print(
                "Using ",
                classification_threshold,
                " for classifying ",
                target,
                " data.",
            )
            print("Converting target data into 1 and 0.")
        all_targets = []

        # TODO:make an all key in qm9_dgl
        if dataset == "qm9_dgl" and target == "all":
            print("Making all qm9_dgl")
            tmp = []
            for ii in d:
                ii["all"] = [
                    ii["mu"],
                    ii["alpha"],
                    ii["homo"],
                    ii["lumo"],
                    ii["gap"],
                    ii["r2"],
                    ii["zpve"],
                    ii["U0"],
                    ii["U"],
                    ii["H"],
                    ii["G"],
                    ii["Cv"],
                ]
                tmp.append(ii)
            print("Made all qm9_dgl")
            d = tmp
        for i in d:
            if isinstance(i[target], list):  # multioutput target
                all_targets.append(torch.tensor(i[target]))
                dat.append(i)

            elif (
                i[target] is not None
                and i[target] != "na"
                and not math.isnan(i[target])
            ):
                if target_multiplication_factor is not None:
                    i[target] = i[target] * target_multiplication_factor
                if classification_threshold is not None:
                    if i[target] <= classification_threshold:
                        i[target] = 0
                    elif i[target] > classification_threshold:
                        i[target] = 1
                    else:
                        raise ValueError(
                            "Check classification data type.",
                            i[target],
                            type(i[target]),
                        )
                dat.append(i)
                all_targets.append(i[target])


    if mp_id_list is not None:
        if mp_id_list == 'bulk':
            print('using mp bulk dataset')
            with open('./data/bulk_megnet_train.pkl', 'rb') as f:
                dataset_train = pk.load(f)
            with open('./data/bulk_megnet_val.pkl', 'rb') as f:
                dataset_val = pk.load(f)
            with open('./data/bulk_megnet_test.pkl', 'rb') as f:
                dataset_test = pk.load(f)

        if mp_id_list == 'shear':
            print('using mp shear dataset')
            with open('./data/shear_megnet_train.pkl', 'rb') as f:
                dataset_train = pk.load(f)
            with open('./data/shear_megnet_val.pkl', 'rb') as f:
                dataset_val = pk.load(f)
            with open('./data/shear_megnet_test.pkl', 'rb') as f:
                dataset_test = pk.load(f)

    else:
        id_train, id_val, id_test = get_id_train_val_test(
            total_size=len(dat),
            split_seed=split_seed,
            train_ratio=train_ratio,
            val_ratio=val_ratio,
            test_ratio=test_ratio,
            n_train=n_train,
            n_test=n_test,
            n_val=n_val,
            keep_data_order=keep_data_order,
        )
        ids_train_val_test = {}
        ids_train_val_test["id_train"] = [dat[i][id_tag] for i in id_train]
        ids_train_val_test["id_val"] = [dat[i][id_tag] for i in id_val]
        ids_train_val_test["id_test"] = [dat[i][id_tag] for i in id_test]
        dumpjson(
            data=ids_train_val_test,
            filename=os.path.join(output_dir, "ids_train_val_test.json"),
        )
        dataset_train = [dat[x] for x in id_train]
        dataset_val = [dat[x] for x in id_val]
        dataset_test = [dat[x] for x in id_test]

    if standard_scalar_and_pca:
        y_data = [i[target] for i in dataset_train]
        # pipe = Pipeline([('scale', StandardScaler())])
        if not isinstance(y_data[0], list):
            print("Running StandardScalar")
            y_data = np.array(y_data).reshape(-1, 1)
        sc = StandardScaler()

        sc.fit(y_data)
        print("Mean", sc.mean_)
        print("Variance", sc.var_)
        try:
            print("New max", max(y_data))
            print("New min", min(y_data))
        except Exception as exp:
            print(exp)
            pass

        pk.dump(sc, open(os.path.join(output_dir, "sc.pkl"), "wb"))

    if classification_threshold is None:
        try:
            from sklearn.metrics import mean_absolute_error

            print("MAX val:", max(all_targets))
            print("MIN val:", min(all_targets))
            print("MAD:", mean_absolute_deviation(all_targets))
            try:
                f = open(os.path.join(output_dir, "mad"), "w")
                line = "MAX val:" + str(max(all_targets)) + "\n"
                line += "MIN val:" + str(min(all_targets)) + "\n"
                line += (
                    "MAD val:"
                    + str(mean_absolute_deviation(all_targets))
                    + "\n"
                )
                f.write(line)
                f.close()
            except Exception as exp:
                print("Cannot write mad", exp)
                pass
            # Random model precited value
            x_bar = np.mean(np.array([i[target] for i in dataset_train]))
            baseline_mae = mean_absolute_error(
                np.array([i[target] for i in dataset_test]),
                np.array([x_bar for i in dataset_test]),
            )
            print("Baseline MAE:", baseline_mae)
        except Exception as exp:
            print("Data error", exp)
            pass

    train_data, mean_train, std_train = get_pyg_dataset(
        dataset=dataset_train,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="train_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
    )
    val_data,_,_ = get_pyg_dataset(
        dataset=dataset_val,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="val_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        mean_train=mean_train,
        std_train=std_train,
    )
    test_data,_,_ = get_pyg_dataset(
        dataset=dataset_test,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="test_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        mean_train=mean_train,
        std_train=std_train,
    )

    collate_fn = train_data.collate
    if line_graph:
        collate_fn = train_data.collate_line_graph

    # use a regular pytorch dataloader
    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        drop_last=True,
        num_workers=workers,
        pin_memory=pin_memory,
    )

    val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        drop_last=True,
        num_workers=workers,
        pin_memory=pin_memory,
    )

    test_loader = DataLoader(
        test_data,
        batch_size=1,
        shuffle=False,
        collate_fn=collate_fn,
        drop_last=False,
        num_workers=workers,
        pin_memory=pin_memory,
    )
    if save_dataloader:
        torch.save(train_loader, train_sample)
        torch.save(val_loader, val_sample)
        torch.save(test_loader, test_sample)

    print("n_train:", len(train_loader.dataset))
    print("n_val:", len(val_loader.dataset))
    print("n_test:", len(test_loader.dataset))
    return (
        train_loader,
        val_loader,
        test_loader,
        train_loader.dataset.prepare_batch,
        mean_train,
        std_train,
    )


In [ ]:
class Matformer(nn.Module):
    def __init__(self, config: MatformerConfig = MatformerConfig(name="matformer")):
        super().__init__()
        self.classification = config.classification
        self.use_angle = config.use_angle
        self.atom_embedding = nn.Linear(config.atom_input_features, config.node_features)

        self.rbf = nn.Sequential(
            RBFExpansion(vmin=0, vmax=8.0, bins=config.edge_features),
            nn.Linear(config.edge_features, config.node_features),
            nn.Softplus(),
            nn.Linear(config.node_features, config.node_features),
        )

        self.angle_lattice = config.angle_lattice

        self.att_layers = nn.ModuleList([
            MatformerConv(
                in_channels=config.node_features,
                out_channels=config.node_features,
                heads=config.node_layer_head,
                edge_dim=config.node_features
            )
            for _ in range(config.conv_layers)
        ])

        self.fc = nn.Sequential(
            nn.Linear(config.node_features, config.fc_features),
            nn.SiLU()
        )

        if self.classification:
            self.fc_out = nn.Linear(config.fc_features, 2)
            self.softmax = nn.LogSoftmax(dim=1)
        else:
            self.fc_out = nn.Linear(config.fc_features, config.output_features)

        self.link_name = config.link
        self.link = self.get_link_fn(config.link, config.zero_inflated)

    def get_link_fn(self, link: str, zero_inflated: bool):
        if link == "identity":
            return lambda x: x
        elif link == "log":
            avg_gap = 0.7
            if not zero_inflated:
                self.fc_out.bias.data = torch.tensor(np.log(avg_gap), dtype=torch.float)
            return torch.exp
        elif link == "logit":
            return torch.sigmoid
        else:
            raise ValueError(f"Unknown link function: {link}")

    def forward(self, data_tuple):
      data, ldata, lattice = data_tuple
      act_list_x = []
      act_list_y = []

      # ---------- Atom Features ----------
      x = self.atom_embedding(data.x)
      atom_feature_avg = torch.mean(x, dim=0, keepdim=True)
      act_list_x.append(atom_feature_avg)
      print("\n[Atom Features] shape:", x.shape)
      print("[Atom Features] sample:\n", x[:5])

      # ---------- Bond Lengths ----------
      bondlength = torch.norm(data.edge_attr, dim=1)
      print("\n[Bond Lengths] shape:", bondlength.shape)
      print("[Bond Lengths] sample:\n", bondlength[:10])

      # ---------- Bond Features ----------
      y = self.rbf(bondlength)
      bond_feature_avg = torch.mean(y, dim=0, keepdim=True)
      act_list_y.append(bond_feature_avg)
      print("\n[Bond Features] shape:", y.shape)
      print("[Bond Features] sample:\n", y[:5])

      # ---------- Attention Layers ----------
      for i, layer in enumerate(self.att_layers):
          print(f"\n[Attention Layer {i + 1}]")
          x = layer(x, data.edge_index, y)
          act_list_x.append(torch.mean(x, dim=0, keepdim=True))
          act_list_y.append(torch.mean(y, dim=0, keepdim=True))

      # ---------- Graph-level Output ----------
      features = scatter(x, data.batch, dim=0, reduce="mean")
      features = self.fc(features)
      out = self.fc_out(features)

      if self.link:
          out = self.link(out)

      if self.classification:
          out = self.softmax(out)

      return torch.squeeze(out), act_list_x, act_list_y



In [ ]:
config = MatformerConfig(name="matformer", conv_layers=4)
model = Matformer(config)

outputs, act_list_x, act_list_y = model((data, None, None))



[Atom Features] shape: torch.Size([2, 128])
[Atom Features] sample:
 tensor([[-0.2976, -0.5521,  0.3740,  0.1117,  0.5190, -1.5470, -0.7916, -0.2023,
         -0.1375,  0.1367,  0.4065, -0.0855,  0.4956, -0.3873,  0.4732, -0.4463,
          0.1207,  0.6740,  0.4839,  0.2251, -0.1911,  0.3847, -0.2233,  0.4441,
          0.9409, -0.5192, -0.1019,  0.9338, -0.2131, -0.1071,  0.3965,  0.2450,
          0.4077, -0.6257,  1.0619, -0.2559, -0.3063,  0.0253,  0.3726,  0.3966,
         -0.7921, -0.0502,  0.6373,  0.3175,  0.0231, -0.1748, -0.7618, -0.1238,
         -0.7776,  0.1801, -0.9898,  0.7548,  0.6404,  0.8304,  0.2705, -0.4908,
         -0.0456,  0.2351,  0.6243, -0.0221, -0.1272,  0.7097,  0.7599,  0.7663,
         -0.2221, -0.7154,  0.0112,  0.0707, -0.7113,  0.7705, -0.3521, -0.3462,
          0.4180, -1.1179,  0.1312, -0.3735,  0.6422, -0.4409, -0.1850,  0.8141,
          0.0146, -0.6391, -0.0435, -0.4133,  0.8307, -0.6684,  0.9834, -0.0545,
         -0.0526,  0.2228,  0.4522,  0.

In [ ]:
print("Loading dataset...")
df = load_dataset(name=dataset_name, target=target_property, limit=1000)
print(f"Loaded {len(df)} samples.")

data = create_graphs(df, target_property)
print(f"Graphs created: {len(data)}")
if len(data) == 0:
    raise ValueError("No valid graphs created.")

# Print node feature dimension of first sample
print("Example node features shape:", data[0].x.shape)  # e.g. torch.Size([N, F])
print("Example edge_attr shape:", data[0].edge_attr.shape)

# Extract actual node feature size
node_feature_dim = data[0].x.shape[1]

# Update config with actual node feature size
config = MatformerConfig(
    name="matformer",
    conv_layers=5,
    node_features=128,
    edge_features=128,
    output_features=1,
    node_layer_head=4,
    atom_input_features=node_feature_dim,  # fix here
)
model = Matformer(config).to(device)


Loading dataset...
Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
Loaded 1000 samples.


100%|██████████| 1000/1000 [00:00<00:00, 1119.67it/s]


Graphs created: 1000
Example node features shape: torch.Size([8, 1])
Example edge_attr shape: torch.Size([56, 1])


  Friday
  

In [4]:
!git clone https://github.com/zjuKeLiu/DPF.git

fatal: destination path 'DPF' already exists and is not an empty directory.


In [5]:
pwd

'/content'

In [6]:
%cd DPF

/content/DPF


graph.py


In [7]:
"""Module to generate networkx graphs."""
"""Implementation based on the template of ALIGNN."""
from multiprocessing.context import ForkContext
from re import X
import numpy as np
import pandas as pd
from jarvis.core.specie import chem_data, get_node_attributes

# from jarvis.core.atoms import Atoms
from collections import defaultdict
from typing import List, Tuple, Sequence, Optional
import torch
from torch_geometric.data import Data
from torch_geometric.transforms import LineGraph
from torch_geometric.data.batch import Batch
import itertools
from jarvis.core.specie import chem_data, get_node_attributes

try:
    import torch
    from tqdm import tqdm
except Exception as exp:
    print("torch/tqdm is not installed.", exp)
    pass

chemical_symbols = [
    # 0
    'X',
    # 1
    'H', 'He',
    # 2
    'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
    # 3
    'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar',
    # 4
    'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
    # 5
    'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
    'In', 'Sn', 'Sb', 'Te', 'I', 'Xe',
    # 6
    'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy',
    'Ho', 'Er', 'Tm', 'Yb', 'Lu',
    'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi',
    'Po', 'At', 'Rn',
    # 7
    'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk',
    'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr',
    'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc',
    'Lv', 'Ts', 'Og']

# pyg dataset
class PygStructureDataset(torch.utils.data.Dataset):
    """Dataset of crystal DGLGraphs."""

    def __init__(
        self,
        df: pd.DataFrame,
        graphs: Sequence[Data],
        target: str,
        atom_features="atomic_number",
        transform=None,
        line_graph=False,
        classification=False,
        id_tag="jid",
        neighbor_strategy="",
        lineControl=True,
        mean_train=None,
        std_train=None,
        pre_train=False,
        masks=None,
        targets_mlm=None,
        targets_lattice=None,
        targets_position=None,
    ):
        """Pytorch Dataset for atomistic graphs.

        `df`: pandas dataframe from e.g. jarvis.db.figshare.data
        `graphs`: DGLGraph representations corresponding to rows in `df`
        `target`: key for label column in `df`
        """
        self.masks = masks
        self.targets_mlm = targets_mlm,
        self.targets_lattice=targets_lattice,
        self.targets_position=targets_position,
        self.df = df
        self.graphs = graphs
        self.target = target
        self.line_graph = line_graph
        self.pre_train = pre_train
        self.ids = self.df[id_tag]
        self.atoms = self.df['atoms']
        #print(self.df.head(2))
        #print("##########################################")
        #print(target)
        #print("##########################################")
        self.labels = torch.tensor(np.array(self.df[target].values)).type(
            torch.get_default_dtype()
        )
        print("mean %f std %f"%(self.labels.mean(), self.labels.std()))
        if mean_train == None:
            mean = self.labels.mean()
            std = self.labels.std()
            self.labels = (self.labels - mean) / std
            print("normalize using training mean but shall not be used here %f and std %f" % (mean, std))
        else:
            self.labels = (self.labels - mean_train) / std_train
            print("normalize using training mean %f and std %f" % (mean_train, std_train))

        self.transform = transform

        features = self._get_attribute_lookup(atom_features)

        # load selected node representation
        # assume graphs contain atomic number in g.ndata["atom_features"]

        '''
        for g in graphs:
            z = g.x
            g.atomic_number = z
            z = z.type(torch.IntTensor).squeeze()
            f = torch.tensor(features[z]).type(torch.FloatTensor)
            if g.x.size(0) == 1:
                f = f.unsqueeze(0)
            g.x = f
        '''
        self.prepare_batch = prepare_pyg_batch
        if line_graph and pre_train:
            self.prepare_batch = prepare_pyg_line_graph_batch_pre_train
            print("pre_train")
            if lineControl == False:
                self.line_graphs = []
                self.graphs = []
                for g in tqdm(graphs):
                    linegraph_trans = LineGraph(force_directed=True)
                    g_new = Data()
                    g_new.x, g_new.edge_index, g_new.edge_attr = g.x, g.edge_index, g.edge_attr
                    try:
                        lg = linegraph_trans(g)
                    except Exception as exp:
                        print(g.x, g.edge_attr, exp)
                        pass
                    lg.edge_attr = pyg_compute_bond_cosines(lg) # old cosine emb
                    # lg.edge_attr = pyg_compute_bond_angle(lg)
                    self.graphs.append(g_new)
                    self.line_graphs.append(lg)
            else:
                if neighbor_strategy == "pairwise-k-nearest":
                    self.graphs = []
                    labels = []
                    idx_t = 0
                    filter_out = 0
                    max_size = 0
                    for g in tqdm(graphs):
                        g.edge_attr = g.edge_attr.float()
                        if g.x.size(0) > max_size:
                            max_size = g.x.size(0)
                        if g.x.size(0) < 200:
                            self.graphs.append(g)
                            labels.append(self.labels[idx_t])
                        else:
                            filter_out += 1
                        idx_t += 1
                    print("filter out %d samples because of exceeding threshold of 200 for nn based method" % filter_out)
                    print("dataset max atom number %d" % max_size)
                    self.line_graphs = self.graphs
                    self.labels = labels
                    self.labels = torch.tensor(self.labels).type(
                                    torch.get_default_dtype()
                                )
                else:
                    self.graphs = []
                    for g in tqdm(graphs):
                        g.edge_attr = g.edge_attr.float()
                        self.graphs.append(g)
                    self.line_graphs = self.graphs
        elif line_graph:
            self.prepare_batch = prepare_pyg_line_graph_batch
            print("building line graphs")
            if lineControl == False:
                self.line_graphs = []
                self.graphs = []
                for g in tqdm(graphs):
                    linegraph_trans = LineGraph(force_directed=True)
                    g_new = Data()
                    g_new.x, g_new.edge_index, g_new.edge_attr = g.x, g.edge_index, g.edge_attr
                    try:
                        lg = linegraph_trans(g)
                    except Exception as exp:
                        print(g.x, g.edge_attr, exp)
                        pass
                    lg.edge_attr = pyg_compute_bond_cosines(lg) # old cosine emb
                    # lg.edge_attr = pyg_compute_bond_angle(lg)
                    self.graphs.append(g_new)
                    self.line_graphs.append(lg)
            else:
                if neighbor_strategy == "pairwise-k-nearest":
                    self.graphs = []
                    labels = []
                    idx_t = 0
                    filter_out = 0
                    max_size = 0
                    for g in tqdm(graphs):
                        g.edge_attr = g.edge_attr.float()
                        if g.x.size(0) > max_size:
                            max_size = g.x.size(0)
                        if g.x.size(0) < 200:
                            self.graphs.append(g)
                            labels.append(self.labels[idx_t])
                        else:
                            filter_out += 1
                        idx_t += 1
                    print("filter out %d samples because of exceeding threshold of 200 for nn based method" % filter_out)
                    print("dataset max atom number %d" % max_size)
                    self.line_graphs = self.graphs
                    self.labels = labels
                    self.labels = torch.tensor(self.labels).type(
                                    torch.get_default_dtype()
                                )
                else:
                    self.graphs = []
                    for g in tqdm(graphs):
                        g.edge_attr = g.edge_attr.float()
                        self.graphs.append(g)
                    self.line_graphs = self.graphs


        if classification:
            self.labels = self.labels.view(-1).long()
            print("Classification dataset.", self.labels)

    @staticmethod
    def _get_attribute_lookup(atom_features: str = "cgcnn"):
        """Build a lookup array indexed by atomic number."""
        max_z = max(v["Z"] for v in chem_data.values())

        # get feature shape (referencing Carbon)
        template = get_node_attributes("C", atom_features)

        features = np.zeros((1 + max_z, len(template)))

        for element, v in chem_data.items():
            z = v["Z"]
            x = get_node_attributes(element, atom_features)

            if x is not None:
                features[z, :] = x

        return features

    def __len__(self):
        """Get length."""
        return self.labels.shape[0]

    def __getitem__(self, idx):
        """Get StructureDataset sample."""
        g = self.graphs[idx]
        label = self.labels[idx]
        return_dict={}
        if self.targets_mlm[0] is not None:
            return_dict["mask"] = self.masks[idx]
            return_dict["atoms"] = self.targets_mlm[0][idx]
        if self.targets_lattice[0] is not None:
            return_dict["lattice"] = self.targets_lattice[0][idx]
        if self.targets_position[0] is not None:
            return_dict["positions"] = self.targets_position[0][idx].t()

        if self.transform:
            g = self.transform(g)

        if self.pre_train and self.line_graph:
            return g, self.line_graphs[idx], return_dict
        elif self.pre_train:
            return g, self.masks[idx], self.targets_mlm[0][idx]

        if self.line_graph:
            return g, self.line_graphs[idx], label, label

        return g, label

    def setup_standardizer(self, ids):
        """Atom-wise feature standardization transform."""
        x = torch.cat(
            [
                g.x
                for idx, g in enumerate(self.graphs)
                if idx in ids
            ]
        )
        self.atom_feature_mean = x.mean(0)
        self.atom_feature_std = x.std(0)

        self.transform = PygStandardize(
            self.atom_feature_mean, self.atom_feature_std
        )

    @staticmethod
    def collate(samples: List[Tuple[Data, torch.Tensor]]):
        """Dataloader helper to batch graphs cross `samples`."""
        graphs, labels = map(list, zip(*samples))
        batched_graph = Batch.from_data_list(graphs)
        return batched_graph, torch.tensor(labels)

    @staticmethod
    def collate_line_graph(
        samples: List[Tuple[Data, Data, torch.Tensor, torch.Tensor]]
    ):
        """Dataloader helper to batch graphs cross `samples`."""
        graphs, line_graphs, lattice, labels = map(list, zip(*samples))
        batched_graph = Batch.from_data_list(graphs)
        batched_line_graph = Batch.from_data_list(line_graphs)
        if len(labels[0].size()) > 0:
            return batched_graph, batched_line_graph, torch.cat([i.unsqueeze(0) for i in lattice]), torch.stack(labels)
        else:
            return batched_graph, batched_line_graph, torch.cat([i.unsqueeze(0) for i in lattice]), torch.tensor(labels)

    @staticmethod
    def collate_line_graph_pretrain(
        samples: List[Tuple[Data, Data, dict]]
    ):
        graphs, line_graphs, return_dict = map(list, zip(*samples))
        batched_graph = Batch.from_data_list(graphs)
        batched_line_graph = Batch.from_data_list(line_graphs)
        target_dict = {}
        if "mask" in return_dict[0]:
            target_mask = []
            target_atom = []
            target_dict["mask"] = target_mask
            target_dict["atoms"] = target_atom
        if "positions" in return_dict[0]:
            target_position = []
            target_dict["positions"] = target_position
        if "lattice" in return_dict[0]:
            target_lattice = []
            target_dict["lattice"] = target_lattice
        for key in target_dict.keys():
            for item in return_dict:
                #print(key, type(item[key]))
                target_dict[key].append(item[key])
            target_dict[key] = torch.hstack(target_dict[key])
        return batched_graph, batched_line_graph, target_dict


def canonize_edge(
    src_id,
    dst_id,
    src_image,
    dst_image,
):
    """Compute canonical edge representation.

    Sort vertex ids
    shift periodic images so the first vertex is in (0,0,0) image
    """
    # store directed edges src_id <= dst_id
    if dst_id < src_id:
        src_id, dst_id = dst_id, src_id
        src_image, dst_image = dst_image, src_image

    # shift periodic images so that src is in (0,0,0) image
    if not np.array_equal(src_image, (0, 0, 0)):
        shift = src_image
        src_image = tuple(np.subtract(src_image, shift))
        dst_image = tuple(np.subtract(dst_image, shift))

    assert src_image == (0, 0, 0)

    return src_id, dst_id, src_image, dst_image


def nearest_neighbor_edges_submit(
    atoms=None,
    cutoff=8,
    max_neighbors=12,
    id=None,
    use_canonize=False,
    use_lattice=False,
    use_angle=False,
):
    """Construct k-NN edge list."""
    # returns List[List[Tuple[site, distance, index, image]]]
    lat = atoms.lattice
    all_neighbors = atoms.get_all_neighbors(r=cutoff)
    min_nbrs = min(len(neighborlist) for neighborlist in all_neighbors)

    attempt = 0
    if min_nbrs < max_neighbors:
        lat = atoms.lattice
        if cutoff < max(lat.a, lat.b, lat.c):
            r_cut = max(lat.a, lat.b, lat.c)
        else:
            r_cut = 2 * cutoff
        attempt += 1
        return nearest_neighbor_edges_submit(
            atoms=atoms,
            use_canonize=use_canonize,
            cutoff=r_cut,
            max_neighbors=max_neighbors,
            id=id,
        )

    edges = defaultdict(set)
    for site_idx, neighborlist in enumerate(all_neighbors):

        # sort on distance
        neighborlist = sorted(neighborlist, key=lambda x: x[2])
        distances = np.array([nbr[2] for nbr in neighborlist])
        ids = np.array([nbr[1] for nbr in neighborlist])
        images = np.array([nbr[3] for nbr in neighborlist])

        # find the distance to the k-th nearest neighbor
        max_dist = distances[max_neighbors - 1]
        ids = ids[distances <= max_dist]
        images = images[distances <= max_dist]
        distances = distances[distances <= max_dist]
        for dst, image in zip(ids, images):
            src_id, dst_id, src_image, dst_image = canonize_edge(
                site_idx, dst, (0, 0, 0), tuple(image)
            )
            if use_canonize:
                edges[(src_id, dst_id)].add(dst_image)
            else:
                edges[(site_idx, dst)].add(tuple(image))

        if use_lattice:
            edges[(site_idx, site_idx)].add(tuple(np.array([0, 0, 1])))
            edges[(site_idx, site_idx)].add(tuple(np.array([0, 1, 0])))
            edges[(site_idx, site_idx)].add(tuple(np.array([1, 0, 0])))
            edges[(site_idx, site_idx)].add(tuple(np.array([0, 1, 1])))
            edges[(site_idx, site_idx)].add(tuple(np.array([1, 0, 1])))
            edges[(site_idx, site_idx)].add(tuple(np.array([1, 1, 0])))

    return edges



def pair_nearest_neighbor_edges(
        atoms=None,
        pair_wise_distances=6,
        use_lattice=False,
        use_angle=False,
):
    """Construct pairwise k-fully connected edge list."""
    smallest = pair_wise_distances
    lattice_list = torch.as_tensor(
        [[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 0], [1, 0, 1], [0, 1, 1]]).float()

    lattice = torch.as_tensor(atoms.lattice_mat).float()
    pos = torch.as_tensor(atoms.cart_coords)
    atom_num = pos.size(0)
    lat = atoms.lattice
    radius_needed = min(lat.a, lat.b, lat.c) * (smallest / 2 - 1e-9)
    r_a = (np.floor(radius_needed / lat.a) + 1).astype(np.int)
    r_b = (np.floor(radius_needed / lat.b) + 1).astype(np.int)
    r_c = (np.floor(radius_needed / lat.c) + 1).astype(np.int)
    period_list = np.array([l for l in itertools.product(*[list(range(-r_a, r_a + 1)), list(range(-r_b, r_b + 1)), list(range(-r_c, r_c + 1))])])
    period_list = torch.as_tensor(period_list).float()
    n_cells = period_list.size(0)
    offset = torch.matmul(period_list, lattice).view(n_cells, 1, 3)
    expand_pos = (pos.unsqueeze(0).expand(n_cells, -1, -1) + offset).transpose(0, 1).contiguous()
    dist = (pos.unsqueeze(1).unsqueeze(1) - expand_pos.unsqueeze(0))  # [n, 1, 1, 3] - [1, n, n_cell, 3] -> [n, n, n_cell, 3]
    dist2, index = torch.sort(dist.norm(dim=-1), dim=-1, stable=True)
    max_value = dist2[:, :, smallest - 1]  # [n, n]
    mask = (dist.norm(dim=-1) <= max_value.unsqueeze(-1))  # [n, n, n_cell]
    shift = torch.matmul(lattice_list, lattice).repeat(atom_num, 1)
    shift_src = torch.arange(atom_num).unsqueeze(-1).repeat(1, lattice_list.size(0))
    shift_src = torch.cat([shift_src[i,:] for i in range(shift_src.size(0))])

    indices = torch.where(mask)
    dist_target = dist[indices]
    u, v, _ = indices
    if use_lattice:
        u = torch.cat((u, shift_src), dim=0)
        v = torch.cat((v, shift_src), dim=0)
        dist_target = torch.cat((dist_target, shift), dim=0)
        assert u.size(0) == dist_target.size(0)

    return u, v, dist_target

def build_undirected_edgedata(
    atoms=None,
    edges={},
):
    """Build undirected graph data from edge set.

    edges: dictionary mapping (src_id, dst_id) to set of dst_image
    r: cartesian displacement vector from src -> dst
    """
    # second pass: construct *undirected* graph
    # import pprint
    u, v, r = [], [], []
    for (src_id, dst_id), images in edges.items():

        for dst_image in images:
            # fractional coordinate for periodic image of dst
            dst_coord = atoms.frac_coords[dst_id] + dst_image
            # cartesian displacement vector pointing from src -> dst
            d = atoms.lattice.cart_coords(
                dst_coord - atoms.frac_coords[src_id]
            )
            # if np.linalg.norm(d)!=0:
            # print ('jv',dst_image,d)
            # add edges for both directions
            for uu, vv, dd in [(src_id, dst_id, d), (dst_id, src_id, -d)]:
                u.append(uu)
                v.append(vv)
                r.append(dd)

    u = torch.tensor(u)
    v = torch.tensor(v)
    if not isinstance(r, list):
        r = torch.tensor(r).type(torch.get_default_dtype())
    else:
        r = torch.tensor(np.array(r)).type(torch.get_default_dtype())
    return u, v, r


class PygGraph(object):
    """Generate a graph object."""

    def __init__(
        self,
        nodes=[],
        node_attributes=[],
        edges=[],
        edge_attributes=[],
        color_map=None,
        labels=None,
    ):
        """
        Initialize the graph object.

        Args:
            nodes: IDs of the graph nodes as integer array.

            node_attributes: node features as multi-dimensional array.

            edges: connectivity as a (u,v) pair where u is
                   the source index and v the destination ID.

            edge_attributes: attributes for each connectivity.
                             as simple as euclidean distances.
        """
        self.nodes = nodes
        self.node_attributes = node_attributes
        self.edges = edges
        self.edge_attributes = edge_attributes
        self.color_map = color_map
        self.labels = labels

    @staticmethod
    def atom_dgl_multigraph(
        atoms=None,
        neighbor_strategy="k-nearest",
        cutoff=8.0,
        max_neighbors=12,
        atom_features="cgcnn",
        max_attempts=3,
        id: Optional[str] = None,
        compute_line_graph: bool = True,
        use_canonize: bool = False,
        use_lattice: bool = False,
        use_angle: bool = False,
    ):
        if neighbor_strategy == "k-nearest":
            edges = nearest_neighbor_edges_submit(
                atoms=atoms,
                cutoff=cutoff,
                max_neighbors=max_neighbors,
                id=id,
                use_canonize=use_canonize,
                use_lattice=use_lattice,
                use_angle=use_angle,
            )
            u, v, r = build_undirected_edgedata(atoms, edges)
        elif neighbor_strategy == "pairwise-k-nearest":
            u, v, r = pair_nearest_neighbor_edges(
                atoms=atoms,
                pair_wise_distances=2,
                use_lattice=use_lattice,
                use_angle=use_angle,
            )
        else:
            raise ValueError("Not implemented yet", neighbor_strategy)


        # build up atom attribute tensor
        sps_features = []
        '''
        for ii, s in enumerate(atoms.elements):
            feat = list(get_node_attributes(s, atom_features=atom_features))
            sps_features.append(feat)
        '''
        '''
        for ii, s in enumerate(atoms.elements):
            one_hot = np.zeros(119)
            one_hot[chemical_symbols.index(s)] = 1
            feat = list(one_hot)
            sps_features.append(feat)

        sps_features = np.array(sps_features)

        node_features = torch.tensor(sps_features).type(
            torch.get_default_dtype()
        )
        '''
        sps_features = []
        for ii, s in enumerate(atoms.elements):
            if s == 'X':

                feat = [0.0] * 92
            else:

                feat = list(get_node_attributes(s, atom_features=atom_features))

                if len(feat) < 92:
                    feat += [0.0] * (92 - len(feat))
                elif len(feat) > 92:
                    feat = feat[:92]
            sps_features.append(feat)

        sps_features = np.array(sps_features)
        node_features = torch.tensor(sps_features).type(
            torch.get_default_dtype()
        )

        edge_index = torch.cat((u.unsqueeze(0), v.unsqueeze(0)), dim=0).long()
        g = Data(x=node_features, edge_index=edge_index, edge_attr=r)

        if compute_line_graph:
            linegraph_trans = LineGraph(force_directed=True)
            g_new = Data()
            g_new.x, g_new.edge_index, g_new.edge_attr = g.x, g.edge_index, g.edge_attr
            lg = linegraph_trans(g)
            lg.edge_attr = pyg_compute_bond_cosines(lg)
            return g_new, lg
        else:
            return g

def pyg_compute_bond_cosines(lg):
    """Compute bond angle cosines from bond displacement vectors."""
    # line graph edge: (a, b), (b, c)
    # `a -> b -> c`
    # use law of cosines to compute angles cosines
    # negate src bond so displacements are like `a <- b -> c`
    # cos(theta) = ba \dot bc / (||ba|| ||bc||)
    src, dst = lg.edge_index
    x = lg.x
    r1 = -x[src]
    r2 = x[dst]
    bond_cosine = torch.sum(r1 * r2, dim=1) / (
        torch.norm(r1, dim=1) * torch.norm(r2, dim=1)
    )
    bond_cosine = torch.clamp(bond_cosine, -1, 1)
    return bond_cosine

def pyg_compute_bond_angle(lg):
    """Compute bond angle from bond displacement vectors."""
    # line graph edge: (a, b), (b, c)
    # `a -> b -> c`
    src, dst = lg.edge_index
    x = lg.x
    r1 = -x[src]
    r2 = x[dst]
    a = (r1 * r2).sum(dim=-1) # cos_angle * |pos_ji| * |pos_jk|
    b = torch.cross(r1, r2).norm(dim=-1) # sin_angle * |pos_ji| * |pos_jk|
    angle = torch.atan2(b, a)
    return angle



class PygStandardize(torch.nn.Module):
    """Standardize atom_features: subtract mean and divide by std."""

    def __init__(self, mean: torch.Tensor, std: torch.Tensor):
        """Register featurewise mean and standard deviation."""
        super().__init__()
        self.mean = mean
        self.std = std

    def forward(self, g: Data):
        """Apply standardization to atom_features."""
        h = g.x
        #g.x = (h - self.mean) / self.std
        return g



def prepare_pyg_batch(
    batch: Tuple[Data, torch.Tensor], device=None, non_blocking=False
):
    """Send batched dgl crystal graph to device."""
    g, t = batch
    batch = (
        g.to(device),
        t.to(device, non_blocking=non_blocking),
    )

    return batch

def prepare_pyg_line_graph_batch_pre_train(
    batch: Tuple[Tuple[Data, Data], dict],
    device=None,
    non_blocking=False,
):
    """Send line graph batch to device.

    Note: the batch is a nested tuple, with the graph and line graph together
    """
    g, lg, return_dict = batch
    for i, item in enumerate(return_dict):
        return_dict[item]  = return_dict[item].to(device, non_blocking=non_blocking)
    batch = (
        (
            g.to(device),
            lg.to(device),
        ),
        (
            return_dict,
        )
    )

    return batch



def prepare_pyg_line_graph_batch(
    batch: Tuple[Tuple[Data, Data, torch.Tensor], torch.Tensor],
    device=None,
    non_blocking=False,
):
    """Send line graph batch to device.

    Note: the batch is a nested tuple, with the graph and line graph together
    """
    g, lg, lattice, t = batch
    batch = (
        (
            g.to(device),
            lg.to(device),
            lattice.to(device, non_blocking=non_blocking),
        ),
        t.to(device, non_blocking=non_blocking),
    )

    return batch

Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.0+cu118


In [9]:
#changed part of the above code
from jarvis.core.specie import chem_data, get_node_attributes
sps_features = []
        for ii, s in enumerate(atoms.elements):
            if s == 'X':

                feat = [0.0] * 92
            else:

                feat = list(get_node_attributes(s, atom_features=atom_features))

                if len(feat) < 92:
                    feat += [0.0] * (92 - len(feat))
                elif len(feat) > 92:
                    feat = feat[:92]
            sps_features.append(feat)

        sps_features = np.array(sps_features)
        node_features = torch.tensor(sps_features).type(
            torch.get_default_dtype()
        )

IndentationError: unexpected indent (<ipython-input-9-d7744a92ea18>, line 4)

data.py

In [10]:
"""Implementation based on the template of ALIGNN."""

import imp
import random
from pathlib import Path
from typing import Optional

# from typing import Dict, List, Optional, Set, Tuple

import os
import torch
import numpy as np
import pandas as pd
from jarvis.core.atoms import Atoms
from matformer.graphs import PygGraph, PygStructureDataset, chemical_symbols
#
from jarvis.db.figshare import data as jdata
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
from jarvis.db.jsonutils import dumpjson

# from sklearn.pipeline import Pipeline
import pickle as pk

from sklearn.preprocessing import StandardScaler
from pandarallel import pandarallel

# use pandas progress_apply
tqdm.pandas()


def load_dataset(
    name: str = "dft_3d",
    target=None,
    limit: Optional[int] = None,
    classification_threshold: Optional[float] = None,
):
    """Load jarvis data."""
    d = jdata(name)
    data = []
    for i in d:
        if i[target] != "na" and not math.isnan(i[target]):
            if classification_threshold is not None:
                if i[target] <= classification_threshold:
                    i[target] = 0
                elif i[target] > classification_threshold:
                    i[target] = 1
                else:
                    raise ValueError(
                        "Check classification data type.",
                        i[target],
                        type(i[target]),
                    )
            data.append(i)
    d = data
    if limit is not None:
        d = d[:limit]
    d = pd.DataFrame(d)
    return d


def mean_absolute_deviation(data, axis=None):
    """Get Mean absolute deviation."""
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)



def load_pyg_graphs(
    df: pd.DataFrame,
    name: str = "dft_3d",
    neighbor_strategy: str = "k-nearest",
    cutoff: float = 8,
    max_neighbors: int = 12,
    cachedir: Optional[Path] = None,
    use_canonize: bool = False,
    use_lattice: bool = False,
    use_angle: bool = False,
):
    """Construct crystal graphs.

    Load only atomic number node features
    and bond displacement vector edge features.

    Resulting graphs have scheme e.g.
    ```
    Graph(num_nodes=12, num_edges=156,
          ndata_schemes={'atom_features': Scheme(shape=(1,)}
          edata_schemes={'r': Scheme(shape=(3,)})
    ```
    """

    def atoms_to_graph(atoms):
        """Convert structure dict to DGLGraph."""
        structure = Atoms.from_dict(atoms)
        return PygGraph.atom_dgl_multigraph(
            structure,
            neighbor_strategy=neighbor_strategy,
            cutoff=cutoff,
            atom_features="atomic_number",
            max_neighbors=max_neighbors,
            compute_line_graph=False,
            use_canonize=use_canonize,
            use_lattice=use_lattice,
            use_angle=use_angle,
        )
    #pandarallel.initialize(progress_bar=True)
    pandarallel.initialize(nb_workers=32)
    graphs = df["atoms"].parallel_apply(atoms_to_graph).values

    return graphs


def get_id_train_val_test(
    total_size=1000,
    split_seed=123,
    train_ratio=None,
    val_ratio=0.1,
    test_ratio=0.1,
    n_train=None,
    n_test=None,
    n_val=None,
    keep_data_order=False,
):
    """Get train, val, test IDs."""
    if (
        train_ratio is None
        and val_ratio is not None
        and test_ratio is not None
    ):
        if train_ratio is None:
            assert val_ratio + test_ratio < 1
            train_ratio = 1 - val_ratio - test_ratio
            print("Using rest of the dataset except the test and val sets.")
        else:
            assert train_ratio + val_ratio + test_ratio <= 1
    # indices = list(range(total_size))
    if n_train is None:
        n_train = int(train_ratio * total_size)
    if n_test is None:
        n_test = int(test_ratio * total_size)
    if n_val is None:
        n_val = int(val_ratio * total_size)
    ids = list(np.arange(total_size))
    if not keep_data_order:
        random.seed(split_seed)
        random.shuffle(ids)
    if n_train + n_val + n_test > total_size:
        raise ValueError(
            "Check total number of samples.",
            n_train + n_val + n_test,
            ">",
            total_size,
        )

    id_train = ids[:n_train]
    id_val = ids[-(n_val + n_test) : -n_test]  # noqa:E203
    id_test = ids[-n_test:]
    return id_train, id_val, id_test


def get_torch_dataset(
    dataset=[],
    id_tag="jid",
    target="",
    neighbor_strategy="",
    atom_features="",
    use_canonize="",
    name="",
    line_graph="",
    cutoff=8.0,
    max_neighbors=12,
    classification=False,
    output_dir=".",
    tmp_name="dataset",
):
    """Get Torch Dataset."""
    df = pd.DataFrame(dataset)
    # print("df", df)
    vals = df[target].values
    if target == "shear modulus" or target == "bulk modulus":
        val_list = [vals[i].item() for i in range(len(vals))]
        vals = val_list
    print("data range", np.max(vals), np.min(vals))
    print("data mean and std", np.mean(vals), np.std(vals))
    f = open(os.path.join(output_dir, tmp_name + "_data_range"), "w")
    line = "Max=" + str(np.max(vals)) + "\n"
    f.write(line)
    line = "Min=" + str(np.min(vals)) + "\n"
    f.write(line)
    f.close()

    graphs = load_graphs(
        df,
        name=name,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
    )

    data = StructureDataset(
        df,
        graphs,
        target=target,
        atom_features=atom_features,
        line_graph=line_graph,
        id_tag=id_tag,
        classification=classification,
    )
    return data

def get_pyg_dataset(
    dataset=[],
    masks = None,
    targets_mlm=None,
    id_tag="jid",
    target="",
    neighbor_strategy="",
    atom_features="",
    use_canonize="",
    name="",
    line_graph="",
    cutoff=8.0,
    max_neighbors=12,
    classification=False,
    output_dir=".",
    tmp_name="dataset",
    use_lattice=False,
    use_angle=False,
    data_from='Jarvis',
    use_save=False,
    mean_train=None,
    std_train=None,
    now=False, # for test
    pre_train=False,
    targets_lattice= None,
    targets_position = None,
):
    """Get pyg Dataset."""
    df = pd.DataFrame(dataset)
    # print("df", df)
    # neighbor_strategy = "pairwise-k-nearest"

    vals = df[target].values
    if target == "shear modulus" or target == "bulk modulus":
        val_list = [vals[i].item() for i in range(len(vals))]
        vals = val_list
    output_dir = "./saved_data/" + tmp_name + "test_graph_angle.pkl" # for fast test use
    print("data range", np.max(vals), np.min(vals))
    print(output_dir)
    if now:
        if not os.path.exists(output_dir):
            graphs = load_pyg_graphs(
                df,
                name=name,
                neighbor_strategy=neighbor_strategy,
                use_canonize=use_canonize,
                cutoff=cutoff,
                max_neighbors=max_neighbors,
                use_lattice=use_lattice,
                use_angle=use_angle,
            )
            with open(output_dir, 'wb') as pf:
                pk.dump(graphs, pf)
            print('save graphs to ', output_dir)
        else:
            print('loading graphs from ', output_dir)
            with open(output_dir, 'rb') as pf:
                graphs = pk.load(pf)
    else:
        print('graphs not saved')
        graphs = load_pyg_graphs(
            df,
            name=name,
            neighbor_strategy=neighbor_strategy,
            use_canonize=use_canonize,
            cutoff=cutoff,
            max_neighbors=max_neighbors,
            use_lattice=use_lattice,
            use_angle=use_angle,
        )
    if mean_train == None:
        mean_train = np.mean(vals)
        std_train = np.std(vals)
        data = PygStructureDataset(
            df,
            graphs,
            masks=masks,
            targets_mlm=targets_mlm,
            targets_lattice=targets_lattice,
            targets_position=targets_position,
            target=target,
            atom_features=atom_features,
            line_graph=line_graph,
            id_tag=id_tag,
            classification=classification,
            neighbor_strategy=neighbor_strategy,
            mean_train=mean_train,
            std_train=std_train,
            pre_train=pre_train
        )
    else:
        data = PygStructureDataset(
            df,
            graphs,
            masks=masks,
            targets_mlm=targets_mlm,
            targets_lattice=targets_lattice,
            targets_position=targets_position,
            target=target,
            atom_features=atom_features,
            line_graph=line_graph,
            id_tag=id_tag,
            classification=classification,
            neighbor_strategy=neighbor_strategy,
            mean_train=mean_train,
            std_train=std_train,
            pre_train=pre_train
        )
    return data, mean_train, std_train


def get_train_val_loaders(
    dataset: str = "dft_3d",
    dataset_array=[],
    target: str = "formation_energy_peratom",
    atom_features: str = "cgcnn",
    neighbor_strategy: str = "k-nearest",
    n_train=None,
    n_val=None,
    n_test=None,
    train_ratio=None,
    val_ratio=0.1,
    test_ratio=0.1,
    batch_size: int = 5,
    standardize: bool = False,
    line_graph: bool = True,
    split_seed: int = 123,
    workers: int = 0,
    pin_memory: bool = True,
    save_dataloader: bool = False,
    filename: str = "sample",
    id_tag: str = "jid",
    use_canonize: bool = False,
    cutoff: float = 8.0,
    max_neighbors: int = 12,
    classification_threshold: Optional[float] = None,
    target_multiplication_factor: Optional[float] = None,
    standard_scalar_and_pca=False,
    keep_data_order=False,
    output_features=1,
    output_dir=None,
    matrix_input=False,
    pyg_input=False,
    use_lattice=False,
    use_angle=False,
    use_save=True,
    mp_id_list=None,
    pre_train=False,
    mask_ratio=None,
    lattice_noise=None,
    position_noise=None,
):
    """Help function to set up JARVIS train and val dataloaders."""
    # data loading
    mean_train=None
    std_train=None
    assert (matrix_input and pyg_input) == False

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    train_sample = os.path.join(output_dir, filename + "_train.data")
    val_sample = os.path.join(output_dir, filename + "_val.data")
    test_sample = os.path.join(output_dir, filename + "_test.data")

    if (
        os.path.exists(train_sample)
        and os.path.exists(val_sample)
        and os.path.exists(test_sample)
        and save_dataloader
    ):
        print("Loading from saved file...")
        print("Make sure all the DataLoader params are same.")
        print("This module is made for debugging only.")
        train_loader = torch.load(train_sample)
        val_loader = torch.load(val_sample)
        test_loader = torch.load(test_sample)
        if train_loader.pin_memory != pin_memory:
            train_loader.pin_memory = pin_memory
        if test_loader.pin_memory != pin_memory:
            test_loader.pin_memory = pin_memory
        if val_loader.pin_memory != pin_memory:
            val_loader.pin_memory = pin_memory
        if train_loader.num_workers != workers:
            train_loader.num_workers = workers
        if test_loader.num_workers != workers:
            test_loader.num_workers = workers
        if val_loader.num_workers != workers:
            val_loader.num_workers = workers
        print("train", len(train_loader.dataset))
        print("val", len(val_loader.dataset))
        print("test", len(test_loader.dataset))
        return (
            train_loader,
            val_loader,
            test_loader,
            train_loader.dataset.prepare_batch,
            None,
            None
        )
    else:
        if not dataset_array:
            d = jdata(dataset)
        else:
            d = dataset_array
            # for ii, i in enumerate(pc_y):
            #    d[ii][target] = pc_y[ii].tolist()

        dat = []
        if classification_threshold is not None:
            print(
                "Using ",
                classification_threshold,
                " for classifying ",
                target,
                " data.",
            )
            print("Converting target data into 1 and 0.")
        all_targets = []

        # TODO:make an all key in qm9_dgl
        if dataset == "qm9_dgl" and target == "all":
            print("Making all qm9_dgl")
            tmp = []
            for ii in d:
                ii["all"] = [
                    ii["mu"],
                    ii["alpha"],
                    ii["homo"],
                    ii["lumo"],
                    ii["gap"],
                    ii["r2"],
                    ii["zpve"],
                    ii["U0"],
                    ii["U"],
                    ii["H"],
                    ii["G"],
                    ii["Cv"],
                ]
                tmp.append(ii)
            print("Made all qm9_dgl")
            d = tmp
        all_targets = []
        masks = []
        position_gt = []
        lattice_gt = []

        for i in d:
            if pre_train:
                if mask_ratio is not None:
                    if len(i["atoms"]["elements"]) < 2:
                        continue
                    num_to_predict = max(1, math.ceil(len(i["atoms"]["elements"])*mask_ratio))
                    sampled_indices = random.sample(range(len(i["atoms"]["elements"])), num_to_predict)
                    atoms = i["atoms"]["elements"]
                    mask = torch.zeros((len(atoms)))
                    z = []
                    for ele in i["atoms"]["elements"]:
                        z.append(chemical_symbols.index(ele))
                    for sampled_index in sampled_indices:
                        i["atoms"]["elements"][sampled_index] = "X"
                        mask[sampled_index] = 1
                    all_targets.append(torch.tensor(np.array(z)))
                    masks.append(mask)
                if position_noise is not None:
                    position_gt.append(torch.tensor(np.array(i['atoms']['coords'])))
                    position_noised = []
                    for position in i['atoms']['coords']:
                        position_noised.append(((np.array(position)+np.random.rand(3)*position_noise)%1.0).tolist())
                    i['atoms']['coords'] = position_noised
                if lattice_noise is not None:
                    lattice_gt.append(torch.tensor(np.array(i['atoms']['lattice_mat'])))
                    lattice_noised = []
                    for lat_vec in i['atoms']['lattice_mat']:
                        lattice_noised.append(((np.array(lat_vec)+np.random.rand(3)*lattice_noise)).tolist())
                    i['atoms']['lattice_mat'] =  lattice_noised
                dat.append(i)

            elif isinstance(i[target], list):  # multioutput target
                all_targets.append(torch.tensor(i[target]))
                dat.append(i)

            elif (
                i[target] is not None
                and i[target] != "na"
                and not math.isnan(i[target])
            ):
                if target_multiplication_factor is not None:
                    i[target] = i[target] * target_multiplication_factor
                if classification_threshold is not None:
                    if i[target] <= classification_threshold:
                        i[target] = 0
                    elif i[target] > classification_threshold:
                        i[target] = 1
                    else:
                        raise ValueError(
                            "Check classification data type.",
                            i[target],
                            type(i[target]),
                        )
                dat.append(i)
                all_targets.append(i[target])


    if mp_id_list is not None:
        if mp_id_list == 'bulk':
            print('using mp bulk dataset')
            with open('./data/bulk_megnet_train.pkl', 'rb') as f:
                dataset_train = pk.load(f)
            with open('./data/bulk_megnet_val.pkl', 'rb') as f:
                dataset_val = pk.load(f)
            with open('./data/bulk_megnet_test.pkl', 'rb') as f:
                dataset_test = pk.load(f)

        if mp_id_list == 'shear':
            print('using mp shear dataset')
            with open('./data/shear_megnet_train.pkl', 'rb') as f:
                dataset_train = pk.load(f)
            with open('./data/shear_megnet_val.pkl', 'rb') as f:
                dataset_val = pk.load(f)
            with open('./data/shear_megnet_test.pkl', 'rb') as f:
                dataset_test = pk.load(f)

    else:
        id_train, id_val, id_test = get_id_train_val_test(
            total_size=len(dat),
            split_seed=split_seed,
            train_ratio=train_ratio,
            val_ratio=val_ratio,
            test_ratio=test_ratio,
            n_train=n_train,
            n_test=n_test,
            n_val=n_val,
            keep_data_order=keep_data_order,
        )
        ids_train_val_test = {}
        ids_train_val_test["id_train"] = [dat[i][id_tag] for i in id_train]
        ids_train_val_test["id_val"] = [dat[i][id_tag] for i in id_val]
        ids_train_val_test["id_test"] = [dat[i][id_tag] for i in id_test]
        dumpjson(
            data=ids_train_val_test,
            filename=os.path.join(output_dir, "ids_train_val_test.json"),
        )
        dataset_train = [dat[x] for x in id_train]
        dataset_val = [dat[x] for x in id_val]
        dataset_test = [dat[x] for x in id_test]
    if pre_train:
        if mask_ratio is not None:
            masks_train = [masks[x] for x in id_train]
            masks_val = [masks[x] for x in id_val]
            masks_test = [masks[x] for x in id_test]
            targets_train = [all_targets[x] for x in id_train]
            targets_val = [all_targets[x] for x in id_val]
            targets_test = [all_targets[x] for x in id_test]
        else:
            targets_train = None
            targets_val = None
            targets_test = None
            masks_train = None
            masks_val = None
            masks_test = None

        if position_noise is not None:
            position_gt_train = [position_gt[x] for x in id_train]
            position_gt_val = [position_gt[x] for x in id_val]
            position_gt_test = [position_gt[x] for x in id_test]
        else:
            position_gt_train = None
            position_gt_val = None
            position_gt_test = None

        if lattice_noise is not None:
            lattice_gt_train = [lattice_gt[x] for x in id_train]
            lattice_gt_val = [lattice_gt[x] for x in id_val]
            lattice_gt_test = [lattice_gt[x] for x in id_test]
        else:
            lattice_gt_train = None
            lattice_gt_val = None
            lattice_gt_test = None


    if pre_train:
        pass
    elif standard_scalar_and_pca:
        y_data = [i[target] for i in dataset_train]
        # pipe = Pipeline([('scale', StandardScaler())])
        if not isinstance(y_data[0], list):
            print("Running StandardScalar")
            y_data = np.array(y_data).reshape(-1, 1)
        sc = StandardScaler()

        sc.fit(y_data)
        print("Mean", sc.mean_)
        print("Variance", sc.var_)
        try:
            print("New max", max(y_data))
            print("New min", min(y_data))
        except Exception as exp:
            print(exp)
            pass

        pk.dump(sc, open(os.path.join(output_dir, "sc.pkl"), "wb"))

    if classification_threshold is None:
        try:
            from sklearn.metrics import mean_absolute_error

            print("MAX val:", max(all_targets))
            print("MIN val:", min(all_targets))
            print("MAD:", mean_absolute_deviation(all_targets))
            try:
                f = open(os.path.join(output_dir, "mad"), "w")
                line = "MAX val:" + str(max(all_targets)) + "\n"
                line += "MIN val:" + str(min(all_targets)) + "\n"
                line += (
                    "MAD val:"
                    + str(mean_absolute_deviation(all_targets))
                    + "\n"
                )
                f.write(line)
                f.close()
            except Exception as exp:
                print("Cannot write mad", exp)
                pass
            # Random model precited value
            x_bar = np.mean(np.array([i[target] for i in dataset_train]))
            baseline_mae = mean_absolute_error(
                np.array([i[target] for i in dataset_test]),
                np.array([x_bar for i in dataset_test]),
            )
            print("Baseline MAE:", baseline_mae)
        except Exception as exp:
            print("Data error", exp)
            pass

    train_data, mean_train, std_train = get_pyg_dataset(
        dataset=dataset_train,
        masks=masks_train,
        targets_mlm= targets_train,
        targets_position = position_gt_train,
        targets_lattice = lattice_gt_train,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="train_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        pre_train=pre_train,
    )
    val_data,_,_ = get_pyg_dataset(
        dataset=dataset_val,
        masks=masks_val,
        targets_mlm= targets_val,
        targets_position = position_gt_val,
        targets_lattice = lattice_gt_val,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="val_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        mean_train=mean_train,
        std_train=std_train,
        pre_train=pre_train,
    )
    test_data,_,_ = get_pyg_dataset(
        dataset=dataset_test,
        masks = masks_test,
        targets_mlm= targets_test,
        targets_position = position_gt_test,
        targets_lattice = lattice_gt_test,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="test_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        mean_train=mean_train,
        std_train=std_train,
        pre_train=pre_train
    )

    collate_fn = train_data.collate
    if pre_train and line_graph:
        collate_fn = train_data.collate_line_graph_pretrain
    elif line_graph:
        collate_fn = train_data.collate_line_graph

    # use a regular pytorch dataloader
    train_loader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        drop_last=False,
        num_workers=workers,
        pin_memory=pin_memory,
    )

    val_loader = DataLoader(
        val_data,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        drop_last=False,
        num_workers=workers,
        pin_memory=pin_memory,
    )

    test_loader = DataLoader(
        test_data,
        batch_size=1,
        shuffle=False,
        collate_fn=collate_fn,
        drop_last=False,
        num_workers=workers,
        pin_memory=pin_memory,
    )
    if save_dataloader:
        torch.save(train_loader, train_sample)
        torch.save(val_loader, val_sample)
        torch.save(test_loader, test_sample)

    print("n_train:", len(train_loader.dataset))
    print("n_val:", len(val_loader.dataset))
    print("n_test:", len(test_loader.dataset))
    return (
        train_loader,
        val_loader,
        test_loader,
        train_loader.dataset.prepare_batch,
        mean_train,
        std_train,
    )


data_accelerator


In [18]:
"""Implementation based on the template of ALIGNN."""

import imp
import random
from pathlib import Path
from typing import Optional
import gc
# from typing import Dict, List, Optional, Set, Tuple

import os
import torch
import numpy as np
import pandas as pd
from jarvis.core.atoms import Atoms
from matformer.graphs import PygGraph, PygStructureDataset, chemical_symbols
#
from jarvis.db.figshare import data as jdata
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
from jarvis.db.jsonutils import dumpjson

# from sklearn.pipeline import Pipeline
import pickle as pk

from sklearn.preprocessing import StandardScaler
from pandarallel import pandarallel
import time
# use pandas progress_apply
tqdm.pandas()

class ConstructTarget():
    def __init__(self, mask_ratio=None, position_noise=None, lattice_noise=None):
        super().__init__()
        self._mask_ratio = mask_ratio
        self._position_noise = position_noise
        self._lattice_noise = lattice_noise
    def forward(self, i):
        pd_dict = {}
        use_this = True
        if self._mask_ratio is not None:
            if len(i["elements"]) < 2:
                use_this = False
                mask = None
                target = None
            else:
                num_to_predict = max(1, math.ceil(len(i["elements"])*self._mask_ratio))
                sampled_indices = random.sample(range(len(i["elements"])), num_to_predict)
                atoms = i["elements"]
                mask = torch.zeros((len(atoms)))
                z = []
                for ele in i["elements"]:
                    z.append(chemical_symbols.index(ele))
                for sampled_index in sampled_indices:
                    i["elements"][sampled_index] = "X"
                    mask[sampled_index] = 1
                target = torch.tensor(np.array(z))
            pd_dict['mask']  = mask
            pd_dict['target_all']  = target
        if self._position_noise is not None:
            pd_dict['position_gt'] = torch.tensor(np.array(i['coords']))
            position_noised = []
            for position in i['coords']:
                position_noised.append(((np.array(position)+np.random.rand(3)*self._position_noise)%1.0).tolist())
            i['coords'] = position_noised
        if self._lattice_noise is not None:
            pd_dict['lattice_gt'] = torch.tensor(np.array(i['lattice_mat']))
            lattice_noised = []
            for lat_vec in i['lattice_mat']:
                lattice_noised.append(((np.array(lat_vec)+np.random.rand(3)*self._lattice_noise)).tolist())
            i['lattice_mat'] =  lattice_noised
        pd_dict['atoms'] = i
        pd_dict['use_this']  = use_this
        #print(len(pd_dict['mask']), len(pd_dict['atoms_noised']["atoms"]["elements"]))
        return pd.Series(pd_dict)

def load_dataset(
    name: str = "dft_3d",
    target=None,
    limit: Optional[int] = None,
    classification_threshold: Optional[float] = None,
):
    """Load jarvis data."""
    d = jdata(name)
    data = []
    for i in d:
        if i[target] != "na" and not math.isnan(i[target]):
            if classification_threshold is not None:
                if i[target] <= classification_threshold:
                    i[target] = 0
                elif i[target] > classification_threshold:
                    i[target] = 1
                else:
                    raise ValueError(
                        "Check classification data type.",
                        i[target],
                        type(i[target]),
                    )
            data.append(i)
    d = data
    if limit is not None:
        d = d[:limit]
    d = pd.DataFrame(d)
    return d


def mean_absolute_deviation(data, axis=None):
    """Get Mean absolute deviation."""
    return np.mean(np.absolute(data - np.mean(data, axis)), axis)


def load_pyg_graphs(
    df: pd.DataFrame,
    name: str = "dft_3d",
    neighbor_strategy: str = "k-nearest",
    cutoff: float = 8,
    max_neighbors: int = 12,
    cachedir: Optional[Path] = None,
    use_canonize: bool = False,
    use_lattice: bool = False,
    use_angle: bool = False,
):
    """Construct crystal graphs.

    Load only atomic number node features
    and bond displacement vector edge features.

    Resulting graphs have scheme e.g.
    ```
    Graph(num_nodes=12, num_edges=156,
          ndata_schemes={'atom_features': Scheme(shape=(1,)}
          edata_schemes={'r': Scheme(shape=(3,)})
    ```
    """

    def atoms_to_graph(atoms):
        """Convert structure dict to DGLGraph."""
        structure = Atoms.from_dict(atoms)
        return PygGraph.atom_dgl_multigraph(
            structure,
            neighbor_strategy=neighbor_strategy,
            cutoff=cutoff,
            atom_features="atomic_number",
            max_neighbors=max_neighbors,
            compute_line_graph=False,
            use_canonize=use_canonize,
            use_lattice=use_lattice,
            use_angle=use_angle,
        )
    #pandarallel.initialize(progress_bar=True)
    pandarallel.initialize()
    graphs = df["atoms"].parallel_apply(atoms_to_graph).values

    return graphs


def get_id_train_val_test(
    total_size=1000,
    split_seed=123,
    train_ratio=None,
    val_ratio=0.1,
    test_ratio=0.1,
    n_train=None,
    n_test=None,
    n_val=None,
    keep_data_order=False,
):
    """Get train, val, test IDs."""
    if (
        train_ratio is None
        and val_ratio is not None
        and test_ratio is not None
    ):
        if train_ratio is None:
            assert val_ratio + test_ratio < 1
            train_ratio = 1 - val_ratio - test_ratio
            print("Using rest of the dataset except the test and val sets.")
        else:
            assert train_ratio + val_ratio + test_ratio <= 1
    # indices = list(range(total_size))
    if n_train is None:
        n_train = int(train_ratio * total_size)
    if n_test is None:
        n_test = int(test_ratio * total_size)
    if n_val is None:
        n_val = int(val_ratio * total_size)
    ids = list(np.arange(total_size))
    if not keep_data_order:
        random.seed(split_seed)
        random.shuffle(ids)
    if n_train + n_val + n_test > total_size:
        raise ValueError(
            "Check total number of samples.",
            n_train + n_val + n_test,
            ">",
            total_size,
        )

    id_train = ids[:n_train]
    id_val = ids[-(n_val + n_test) : -n_test]  # noqa:E203
    id_test = ids[-n_test:]
    return id_train, id_val, id_test


def get_torch_dataset(
    dataset=[],
    id_tag="jid",
    target="",
    neighbor_strategy="",
    atom_features="",
    use_canonize="",
    name="",
    line_graph="",
    cutoff=8.0,
    max_neighbors=12,
    classification=False,
    output_dir=".",
    tmp_name="dataset",
):
    """Get Torch Dataset."""
    df = pd.DataFrame(dataset)
    # print("df", df)
    vals = df[target].values
    if target == "shear modulus" or target == "bulk modulus":
        val_list = [vals[i].item() for i in range(len(vals))]
        vals = val_list
    print("data range", np.max(vals), np.min(vals))
    print("data mean and std", np.mean(vals), np.std(vals))
    f = open(os.path.join(output_dir, tmp_name + "_data_range"), "w")
    line = "Max=" + str(np.max(vals)) + "\n"
    f.write(line)
    line = "Min=" + str(np.min(vals)) + "\n"
    f.write(line)
    f.close()

    graphs = load_graphs(
        df,
        name=name,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
    )

    data = StructureDataset(
        df,
        graphs,
        target=target,
        atom_features=atom_features,
        line_graph=line_graph,
        id_tag=id_tag,
        classification=classification,
    )
    return data

def get_pyg_dataset(
    dataset=[],
    masks = None,
    targets_mlm=None,
    id_tag="jid",
    target="",
    neighbor_strategy="",
    atom_features="",
    use_canonize="",
    name="",
    line_graph="",
    cutoff=8.0,
    max_neighbors=12,
    classification=False,
    output_dir=".",
    tmp_name="dataset",
    use_lattice=False,
    use_angle=False,
    data_from='Jarvis',
    use_save=False,
    mean_train=None,
    std_train=None,
    now=False, # for test
    pre_train=False,
    targets_lattice= None,
    targets_position = None,
):
    """Get pyg Dataset."""
    df = pd.DataFrame(dataset)
    # print("df", df)
    # neighbor_strategy = "pairwise-k-nearest"

    vals = df[target].values
    if target == "shear modulus" or target == "bulk modulus":
        val_list = [vals[i].item() for i in range(len(vals))]
        vals = val_list
    output_dir = "./saved_data/" + tmp_name + "test_graph_angle.pkl" # for fast test use
    print("data range", np.max(vals), np.min(vals))
    print(output_dir)
    if now:
        if not os.path.exists(output_dir):
            graphs = load_pyg_graphs(
                df,
                name=name,
                neighbor_strategy=neighbor_strategy,
                use_canonize=use_canonize,
                cutoff=cutoff,
                max_neighbors=max_neighbors,
                use_lattice=use_lattice,
                use_angle=use_angle,
            )
            with open(output_dir, 'wb') as pf:
                pk.dump(graphs, pf)
            print('save graphs to ', output_dir)
        else:
            print('loading graphs from ', output_dir)
            with open(output_dir, 'rb') as pf:
                graphs = pk.load(pf)
    else:
        print('graphs not saved')
        graphs = load_pyg_graphs(
            df,
            name=name,
            neighbor_strategy=neighbor_strategy,
            use_canonize=use_canonize,
            cutoff=cutoff,
            max_neighbors=max_neighbors,
            use_lattice=use_lattice,
            use_angle=use_angle,
        )
    if mean_train == None:
        mean_train = np.mean(vals)
        std_train = np.std(vals)
        data = PygStructureDataset(
            df,
            graphs,
            masks=masks,
            targets_mlm=targets_mlm,
            targets_lattice=targets_lattice,
            targets_position=targets_position,
            target=target,
            atom_features=atom_features,
            line_graph=line_graph,
            id_tag=id_tag,
            classification=classification,
            neighbor_strategy=neighbor_strategy,
            mean_train=mean_train,
            std_train=std_train,
            pre_train=pre_train
        )
    else:
        data = PygStructureDataset(
            df,
            graphs,
            masks=masks,
            targets_mlm=targets_mlm,
            targets_lattice=targets_lattice,
            targets_position=targets_position,
            target=target,
            atom_features=atom_features,
            line_graph=line_graph,
            id_tag=id_tag,
            classification=classification,
            neighbor_strategy=neighbor_strategy,
            mean_train=mean_train,
            std_train=std_train,
            pre_train=pre_train
        )
    return data, mean_train, std_train


def get_train_val_loaders(
    dataset: str = "dft_3d",
    dataset_array=[],
    target: str = "formation_energy_peratom",
    atom_features: str = "cgcnn",
    neighbor_strategy: str = "k-nearest",
    n_train=None,
    n_val=None,
    n_test=None,
    train_ratio=None,
    val_ratio=0.1,
    test_ratio=0.1,
    batch_size: int = 5,
    standardize: bool = False,
    line_graph: bool = True,
    split_seed: int = 123,
    workers: int = 0,
    pin_memory: bool = True,
    save_dataloader: bool = False,
    filename: str = "sample",
    id_tag: str = "jid",
    use_canonize: bool = False,
    cutoff: float = 8.0,
    max_neighbors: int = 12,
    classification_threshold: Optional[float] = None,
    target_multiplication_factor: Optional[float] = None,
    standard_scalar_and_pca=False,
    keep_data_order=False,
    output_features=1,
    output_dir=None,
    matrix_input=False,
    pyg_input=False,
    use_lattice=False,
    use_angle=False,
    use_save=True,
    mp_id_list=None,
    pre_train=False,
    mask_ratio=None,
    lattice_noise=None,
    position_noise=None,
):
    """Help function to set up JARVIS train and val dataloaders."""
    # data loading
    mean_train=None
    std_train=None
    assert (matrix_input and pyg_input) == False
    #print("*******get_train_val_loaders***********")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    train_sample = os.path.join(output_dir, filename + "_train.data")
    val_sample = os.path.join(output_dir, filename + "_val.data")
    test_sample = os.path.join(output_dir, filename + "_test.data")

    if (
        os.path.exists(train_sample)
        and os.path.exists(val_sample)
        and os.path.exists(test_sample)
        and save_dataloader
    ):
        print("Loading from saved file...")
        print("Make sure all the DataLoader params are same.")
        print("This module is made for debugging only.")
        train_loader = torch.load(train_sample)
        val_loader = torch.load(val_sample)
        test_loader = torch.load(test_sample)
        if train_loader.pin_memory != pin_memory:
            train_loader.pin_memory = pin_memory
        if test_loader.pin_memory != pin_memory:
            test_loader.pin_memory = pin_memory
        if val_loader.pin_memory != pin_memory:
            val_loader.pin_memory = pin_memory
        if train_loader.num_workers != workers:
            train_loader.num_workers = workers
        if test_loader.num_workers != workers:
            test_loader.num_workers = workers
        if val_loader.num_workers != workers:
            val_loader.num_workers = workers
        print("train", len(train_loader.dataset))
        print("val", len(val_loader.dataset))
        print("test", len(test_loader.dataset))
        return (
            train_loader,
            val_loader,
            test_loader,
            train_loader.dataset.prepare_batch,
            None,
            None
        )
    else:
        if not dataset_array:
            d = jdata(dataset)
        else:
            d = dataset_array
            # for ii, i in enumerate(pc_y):
            #    d[ii][target] = pc_y[ii].tolist()

        dat = []
        if classification_threshold is not None:
            print(
                "Using ",
                classification_threshold,
                " for classifying ",
                target,
                " data.",
            )
            print("Converting target data into 1 and 0.")
        all_targets = []

        # TODO:make an all key in qm9_dgl
        if dataset == "qm9_dgl" and target == "all":
            print("Making all qm9_dgl")
            tmp = []
            for ii in d:
                ii["all"] = [
                    ii["mu"],
                    ii["alpha"],
                    ii["homo"],
                    ii["lumo"],
                    ii["gap"],
                    ii["r2"],
                    ii["zpve"],
                    ii["U0"],
                    ii["U"],
                    ii["H"],
                    ii["G"],
                    ii["Cv"],
                ]
                tmp.append(ii)
            print("Made all qm9_dgl")
            d = tmp
        all_targets = []
        masks = []
        position_gt = []
        lattice_gt = []
        construct = ConstructTarget(mask_ratio=mask_ratio, position_noise=position_noise, lattice_noise=lattice_noise)
        print("*******Construct targets***********")
        tc1 = time.time()
        if pre_train:
            #pandarallel.initialize(progress_bar=True)
            pandarallel.initialize()
            df = pd.DataFrame(d)
            #print(df['atoms'][0]['elements'])
            #print(df.head(2))
            data_frame_temp = pd.DataFrame(df['atoms'].parallel_apply(construct.forward))
            data_frame_temp['jid'] = df['jid']
            data_frame_temp['target'] = df['target']
            #print(data_frame_temp.head(2))
            data_frame = data_frame_temp[data_frame_temp['use_this']==True].reindex()
            #for i in range(10):
            #    print(len(data_frame['mask'][i]), len(data_frame['atoms'][i]["elements"]))
            #print(data_frame.head(2))
            dat = data_frame.to_dict('records')
            #dat = list(data_frame['atoms_noised'].values)
            del data_frame_temp
            gc.collect()
        tc2 = time.time()
        print(f"*******Construct targets done{tc2-tc1}s***********")
        '''
        for i in tqdm(d):
            if pre_train:
                if mask_ratio is not None:
                    if len(i["atoms"]["elements"]) < 2:
                        continue
                    num_to_predict = max(1, math.ceil(len(i["atoms"]["elements"])*mask_ratio))
                    sampled_indices = random.sample(range(len(i["atoms"]["elements"])), num_to_predict)
                    atoms = i["atoms"]["elements"]
                    mask = torch.zeros((len(atoms)))
                    z = []
                    for ele in i["atoms"]["elements"]:
                        z.append(chemical_symbols.index(ele))
                    for sampled_index in sampled_indices:
                        i["atoms"]["elements"][sampled_index] = "X"
                        mask[sampled_index] = 1
                    all_targets.append(torch.tensor(np.array(z)))
                    masks.append(mask)
                if position_noise is not None:
                    position_gt.append(torch.tensor(np.array(i['atoms']['coords'])))
                    position_noised = []
                    for position in i['atoms']['coords']:
                        position_noised.append(((np.array(position)+np.random.rand(3)*position_noise)%1.0).tolist())
                    i['atoms']['coords'] = position_noised
                if lattice_noise is not None:
                    lattice_gt.append(torch.tensor(np.array(i['atoms']['lattice_mat'])))
                    lattice_noised = []
                    for lat_vec in i['atoms']['lattice_mat']:
                        lattice_noised.append(((np.array(lat_vec)+np.random.rand(3)*lattice_noise)).tolist())
                    i['atoms']['lattice_mat'] =  lattice_noised
                dat.append(i)
            elif isinstance(i[target], list):  # multioutput target
                all_targets.append(torch.tensor(i[target]))
                dat.append(i)

            elif (
                i[target] is not None
                and i[target] != "na"
                and not math.isnan(i[target])
            ):
                if target_multiplication_factor is not None:
                    i[target] = i[target] * target_multiplication_factor
                if classification_threshold is not None:
                    if i[target] <= classification_threshold:
                        i[target] = 0
                    elif i[target] > classification_threshold:
                        i[target] = 1
                    else:
                        raise ValueError(
                            "Check classification data type.",
                            i[target],
                            type(i[target]),
                        )
                dat.append(i)
                all_targets.append(i[target])
        '''

    if mp_id_list is not None:
        if mp_id_list == 'bulk':
            print('using mp bulk dataset')
            with open('./data/bulk_megnet_train.pkl', 'rb') as f:
                dataset_train = pk.load(f)
            with open('./data/bulk_megnet_val.pkl', 'rb') as f:
                dataset_val = pk.load(f)
            with open('./data/bulk_megnet_test.pkl', 'rb') as f:
                dataset_test = pk.load(f)

        if mp_id_list == 'shear':
            print('using mp shear dataset')
            with open('./data/shear_megnet_train.pkl', 'rb') as f:
                dataset_train = pk.load(f)
            with open('./data/shear_megnet_val.pkl', 'rb') as f:
                dataset_val = pk.load(f)
            with open('./data/shear_megnet_test.pkl', 'rb') as f:
                dataset_test = pk.load(f)

    else:
        id_train, id_val, id_test = get_id_train_val_test(
            total_size=len(dat),
            split_seed=split_seed,
            train_ratio=train_ratio,
            val_ratio=val_ratio,
            test_ratio=test_ratio,
            n_train=n_train,
            n_test=n_test,
            n_val=n_val,
            keep_data_order=keep_data_order,
        )
        ids_train_val_test = {}
        ids_train_val_test["id_train"] = [dat[i][id_tag] for i in id_train]
        ids_train_val_test["id_val"] = [dat[i][id_tag] for i in id_val]
        ids_train_val_test["id_test"] = [dat[i][id_tag] for i in id_test]
        dumpjson(
            data=ids_train_val_test,
            filename=os.path.join(output_dir, "ids_train_val_test.json"),
        )
        dataset_train = [dat[x] for x in id_train]
        dataset_val = [dat[x] for x in id_val]
        dataset_test = [dat[x] for x in id_test]
    if pre_train:
        if mask_ratio is not None:
            print("Get masks")
            masks = data_frame['mask'].values
            print("Get atom types")
            all_targets = data_frame['target_all'].values
            masks_train = masks[id_train]
            masks_val = masks[id_val]
            masks_test = masks[id_test]
            targets_train = all_targets[id_train]
            targets_val = all_targets[id_val]
            targets_test = all_targets[id_test]
        else:
            targets_train = None
            targets_val = None
            targets_test = None
            masks_train = None
            masks_val = None
            masks_test = None

        if position_noise is not None:
            print("Get position GT")
            position_gt = data_frame['position_gt'].values
            position_gt_train = position_gt[id_train]
            position_gt_val = position_gt[id_val]
            position_gt_test = position_gt[id_test]
        else:
            position_gt_train = None
            position_gt_val = None
            position_gt_test = None

        if lattice_noise is not None:
            print("Get lattice gt")
            lattice_gt = data_frame['lattice_gt'].values
            lattice_gt_train = lattice_gt[id_train]
            lattice_gt_val = lattice_gt[id_val]
            lattice_gt_test = lattice_gt[id_test]
        else:
            lattice_gt_train = None
            lattice_gt_val = None
            lattice_gt_test = None
        '''
        if mask_ratio is not None:
            print("Get masks")
            masks = list(data_frame['mask'].values)
            print("Get atom types")
            all_targets = list(data_frame['target'].values)
            masks_train = [masks[x] for x in id_train]
            masks_val = [masks[x] for x in id_val]
            masks_test = [masks[x] for x in id_test]
            targets_train = [all_targets[x] for x in id_train]
            targets_val = [all_targets[x] for x in id_val]
            targets_test = [all_targets[x] for x in id_test]
        else:
            targets_train = None
            targets_val = None
            targets_test = None
            masks_train = None
            masks_val = None
            masks_test = None

        if position_noise is not None:
            print("Get position GT")
            position_gt = list(data_frame['position_gt'].values)
            position_gt_train = [position_gt[x] for x in id_train]
            position_gt_val = [position_gt[x] for x in id_val]
            position_gt_test = [position_gt[x] for x in id_test]
        else:
            position_gt_train = None
            position_gt_val = None
            position_gt_test = None

        if lattice_noise is not None:
            print("Get lattice gt")
            lattice_gt = list(data_frame['lattice_gt'].values)
            lattice_gt_train = [lattice_gt[x] for x in id_train]
            lattice_gt_val = [lattice_gt[x] for x in id_val]
            lattice_gt_test = [lattice_gt[x] for x in id_test]
        else:
            lattice_gt_train = None
            lattice_gt_val = None
            lattice_gt_test = None
        '''

    if pre_train:
        pass
    elif standard_scalar_and_pca:
        y_data = [i[target] for i in dataset_train]
        # pipe = Pipeline([('scale', StandardScaler())])
        if not isinstance(y_data[0], list):
            print("Running StandardScalar")
            y_data = np.array(y_data).reshape(-1, 1)
        sc = StandardScaler()

        sc.fit(y_data)
        print("Mean", sc.mean_)
        print("Variance", sc.var_)
        try:
            print("New max", max(y_data))
            print("New min", min(y_data))
        except Exception as exp:
            print(exp)
            pass

        pk.dump(sc, open(os.path.join(output_dir, "sc.pkl"), "wb"))

    if classification_threshold is None:
        try:
            from sklearn.metrics import mean_absolute_error

            print("MAX val:", max(all_targets))
            print("MIN val:", min(all_targets))
            print("MAD:", mean_absolute_deviation(all_targets))
            try:
                f = open(os.path.join(output_dir, "mad"), "w")
                line = "MAX val:" + str(max(all_targets)) + "\n"
                line += "MIN val:" + str(min(all_targets)) + "\n"
                line += (
                    "MAD val:"
                    + str(mean_absolute_deviation(all_targets))
                    + "\n"
                )
                f.write(line)
                f.close()
            except Exception as exp:
                print("Cannot write mad", exp)
                pass
            # Random model precited value
            x_bar = np.mean(np.array([i[target] for i in dataset_train]))
            baseline_mae = mean_absolute_error(
                np.array([i[target] for i in dataset_test]),
                np.array([x_bar for i in dataset_test]),
            )
            print("Baseline MAE:", baseline_mae)
        except Exception as exp:
            print("Data error", exp)
            pass

    train_data, mean_train, std_train = get_pyg_dataset(
        dataset=dataset_train,
        masks=masks_train,
        targets_mlm= targets_train,
        targets_position = position_gt_train,
        targets_lattice = lattice_gt_train,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="train_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        pre_train=pre_train,
    )
    val_data,_,_ = get_pyg_dataset(
        dataset=dataset_val,
        masks=masks_val,
        targets_mlm= targets_val,
        targets_position = position_gt_val,
        targets_lattice = lattice_gt_val,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="val_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        mean_train=mean_train,
        std_train=std_train,
        pre_train=pre_train,
    )
    test_data,_,_ = get_pyg_dataset(
        dataset=dataset_test,
        masks = masks_test,
        targets_mlm= targets_test,
        targets_position = position_gt_test,
        targets_lattice = lattice_gt_test,
        id_tag=id_tag,
        atom_features=atom_features,
        target=target,
        neighbor_strategy=neighbor_strategy,
        use_canonize=use_canonize,
        name=dataset,
        line_graph=line_graph,
        cutoff=cutoff,
        max_neighbors=max_neighbors,
        classification=classification_threshold is not None,
        output_dir=output_dir,
        tmp_name="test_data",
        use_lattice=use_lattice,
        use_angle=use_angle,
        use_save=False,
        mean_train=mean_train,
        std_train=std_train,
        pre_train=pre_train
    )

    collate_fn = train_data.collate
    if pre_train and line_graph:
        collate_fn = train_data.collate_line_graph_pretrain
    elif line_graph:
        collate_fn = train_data.collate_line_graph
    return (train_data, val_data, test_data, collate_fn, pin_memory)
    # use a regular pytorch dataloader



pyg_att.-py


In [63]:
"""Implementation based on the template of ALIGNN."""

from typing import Tuple, Optional, List
import numpy as np
import torch
import torch.nn.functional as F
from pydantic.typing import Literal
from torch import nn
from matformer.models.utils import RBFExpansion
from matformer.features import angle_emb_mp
from torch_scatter import scatter
from matformer.models.transformer import MatformerConv
from pydantic import BaseSettings as PydanticBaseSettings


class BaseSettings(PydanticBaseSettings):
    """Add configuration to default Pydantic BaseSettings."""
    class Config:
        extra = "forbid"
        use_enum_values = True
        env_prefix = "jv_"


class MatformerConfig(BaseSettings):
    """Hyperparameter schema for Matformer model."""
    name: Literal["matformer"]
    conv_layers: int = 5
    edge_layers: int = 0
    atom_input_features: int = 92
    edge_features: int = 128
    triplet_input_features: int = 40
    node_features: int = 128
    fc_layers: int = 1
    fc_features: int = 128
    output_features: int = 1
    node_layer_head: int = 4
    edge_layer_head: int = 4
    nn_based: bool = False
    link: Literal["identity", "log", "logit"] = "identity"
    zero_inflated: bool = False
    use_angle: bool = False
    angle_lattice: bool = False
    classification: bool = False
    pre_train: bool = False
    position_noise: Optional[float] = None
    lattice_noise: Optional[float] = None
    mask_ratio: Optional[float] = None

    class Config:
        env_prefix = "jv_model"


class Matformer(nn.Module):
    """Matformer: A transformer-based model for materials property prediction."""

    def __init__(self, config: MatformerConfig = MatformerConfig(name="matformer")):
        super().__init__()
        self.config = config
        self.classification = config.classification
        self.use_angle = config.use_angle
        self.pre_train = config.pre_train
        self.zero_inflated = config.zero_inflated
        self.mask_ratio = config.mask_ratio is not None
        self.position_noise = config.position_noise is not None
        self.lattice_noise = config.lattice_noise is not None

        # Atom embedding layer
        self.atom_embedding = nn.Linear(
            config.atom_input_features, config.node_features
        )

        # Radial basis function expansion for edge features
        self.rbf = nn.Sequential(
            RBFExpansion(vmin=0, vmax=8.0, bins=config.edge_features),
            nn.Linear(config.edge_features, config.node_features),
            nn.Softplus(),
            nn.Linear(config.node_features, config.node_features),
        )

        # Attention layers
        self.att_layers = nn.ModuleList([
            MatformerConv(
                in_channels=config.node_features,
                out_channels=config.node_features,
                heads=config.node_layer_head,
                edge_dim=config.node_features
            ) for _ in range(config.conv_layers)
        ])

        # Output layers
        self.fc = nn.Sequential(
            nn.Linear(config.node_features, config.fc_features),
            nn.SiLU()
        )
        self.fc_out = nn.Linear(config.fc_features, config.output_features)

        # Link function
        self.link_name = config.link
        if config.link == "identity":
            self.link = lambda x: x
        elif config.link == "log":
            self.link = torch.exp
            if not self.zero_inflated:
                self.fc_out.bias.data = torch.tensor(np.log(0.7), dtype=torch.float)
        elif config.link == "logit":
            self.link = torch.sigmoid

        # Classification-specific layers
        if self.classification:
            self.softmax = nn.LogSoftmax(dim=1)

        # Pretraining-specific layers
        if self.mask_ratio:
            self.mlm_pred = nn.Linear(config.node_features, 119)
            self.softmax_mlm = nn.LogSoftmax(dim=-1)
        if self.position_noise:
            self.position_mlp = nn.Linear(config.node_features, 3)
        if self.lattice_noise:
            self.lattice_mlp = nn.Linear(config.node_features, 9)

    def forward(self, data_tuple: Tuple) -> torch.Tensor:

        x_list = []
        y_list = []

        """
        Forward pass of the Matformer model.

        Args:
            data_tuple: Tuple containing:
                - data: PyG Data object with node features and edge indices
                - ldata: Line graph data (unused in current implementation)
                - lattice: Lattice information (unused in current implementation)

        Returns:
            torch.Tensor: Model predictions
        """
        data, _, _ = data_tuple  # Unpack the data tuple

        # Atom feature embedding
        x = self.atom_embedding(data.x)
        atom_feature_avg = torch.mean(x, dim=0, keepdim=True)
        x_list.append(atom_feature_avg)
        print("\n[Atom Features] shape:", x.shape)
        print("[Atom Features] sample:\n", x[:5])

        # Edge feature processing
        bondlength = torch.norm(data.edge_attr, dim=1)
        print("\n[Bond Lengths] shape:", bondlength.shape)
        print("[Bond Lengths] sample:\n", bondlength[:10])
        y = self.rbf(bondlength)
        bond_feature_avg = torch.mean(y, dim=0, keepdim=True)
        y_list.append(bond_feature_avg)
        print("\n[Bond Features] shape:", y.shape)
        print("[Bond Features] sample:\n", y[:5])

        # Attention layers
        for layer in self.att_layers:
            x = layer(x, data.edge_index, y)

        # Graph-level readout
        features = scatter(x, data.batch, dim=0, reduce="mean")
        features = self.fc(features)
        out = self.fc_out(features)

        # Apply link function if specified
        if hasattr(self, 'link') and self.link:
            out = self.link(out)

        # Classification-specific processing
        if self.classification:
            out = self.softmax(out)

        return out.squeeze(-1),x_list,y_list  # Remove last dimension if output_features=1


# Example usage:
if __name__ == "__main__":
    config = MatformerConfig(
        name="matformer",
        atom_input_features=92,
        node_features=128,
        output_features=1
    )
    model = Matformer(config)
    print(f"Model architecture:\n{model}")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

Model architecture:
Matformer(
  (atom_embedding): Linear(in_features=92, out_features=128, bias=True)
  (rbf): Sequential(
    (0): RBFExpansion()
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): Softplus(beta=1, threshold=20)
    (3): Linear(in_features=128, out_features=128, bias=True)
  )
  (att_layers): ModuleList(
    (0-4): 5 x MatformerConv(128, 128, heads=4)
  )
  (fc): Sequential(
    (0): Linear(in_features=128, out_features=128, bias=True)
    (1): SiLU()
  )
  (fc_out): Linear(in_features=128, out_features=1, bias=True)
)
Total parameters: 2,782,849


In [ ]:
# Example usage:
if __name__ == "__main__":
    config = MatformerConfig(
        name="matformer",
        atom_input_features=92,
        node_features=128,
        output_features=1
    )
    model = Matformer(config)
    print(f"Model architecture:\n{model}")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Create dummy input data
    from torch_geometric.data import Data

    # Example with 4 atoms and 4 bonds
    num_atoms = 4
    num_bonds = 4

    # Atom features (92-dimensional)
    x = torch.randn(num_atoms, config.atom_input_features)

    # Edge indices (bonds between atoms)
    edge_index = torch.tensor([[0, 1, 2, 3],  # Source nodes
                              [1, 2, 3, 0]], # Target nodes
                             dtype=torch.long)

    # Edge attributes (3D vectors representing bond vectors)
    edge_attr = torch.randn(num_bonds, 3)

    # Batch indices (all atoms belong to same graph)
    batch = torch.zeros(num_atoms, dtype=torch.long)

    # Create PyG Data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, batch=batch)

    # Create the input tuple expected by forward()
    data_tuple = (data, None, None)  # (graph_data, line_graph_data, lattice_data)

    # Call the forward pass
    output, atom_features, bond_features = model.forward(data_tuple)

    # Print the collected features
    print("\n=== Collected Features ===")
    print("\nAtom features (averaged):")
    print(atom_features[0].shape)  # Should be [1, 128]
    print(atom_features[0])

    print("\nBond features (averaged):")
    print(bond_features[0].shape)  # Should be [1, 128]
    print(bond_features[0])

    # Note: If you want to access the actual bond lengths, you'll need to modify
    # the forward() method to return them as well (add bondlength to return tuple)

In [70]:

model = Matformer(config)


out, atom_embeds, bond_embeds = model((data, None, None))




[Atom Features] shape: torch.Size([4, 128])
[Atom Features] sample:
 tensor([[-1.7811e-02, -3.8489e-01,  1.1573e-01, -4.1183e-01, -7.2481e-01,
          9.5015e-01,  9.2779e-01, -4.3211e-01,  1.0670e+00,  8.5227e-02,
          4.7383e-01,  6.7563e-01, -3.6556e-01, -5.6806e-01, -3.2215e-01,
         -6.2905e-01,  2.7742e-01,  1.0455e+00, -6.3027e-01, -6.9554e-01,
          2.2431e-01, -4.4384e-01, -8.4471e-01, -4.6448e-02, -4.9980e-01,
         -7.5410e-01, -8.0779e-01,  2.7518e-01,  2.3975e-01,  4.2234e-01,
         -6.3257e-01,  3.1074e-01, -5.0124e-01,  1.0280e+00,  3.1776e-01,
         -5.5841e-01,  2.5053e-01,  2.9950e-01,  2.9211e-01,  6.9621e-01,
         -1.5181e-01, -1.2056e-01,  6.8484e-02,  3.4270e-01,  6.7564e-03,
         -9.4143e-01,  3.3730e-02,  9.1563e-02, -1.9037e-01,  5.0819e-02,
          4.8274e-01, -8.1192e-01,  5.8583e-02, -1.2357e-01,  2.4800e-01,
          7.0231e-01,  3.4856e-01,  3.8016e-01,  8.9545e-01, -7.0316e-01,
          6.5440e-01,  5.8041e-02,  3.9808

In [17]:
!python3

Python 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> ^C


In [82]:
cd matformer/

[Errno 2] No such file or directory: 'matformer/'
/content/DPF/matformer


In [72]:
!sh pretrain.sh

Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.0+cu118
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `0`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.0+cu118
name='matformer' conv_layers=5 edge_layers=0 atom_input_features=92 edge_features=128 triplet_input_features=40 node_features=128 fc_layers=1 fc_features=128 output_features=1 node_layer_head=4 edge_layer_head=4 nn_based=False link='identity' zero_inflated=False use_angle=False angle_lattice=False classification=False pre_train=False position_noise=None lattice_noise=None mask_ratio=None
{'dataset': 'user_data', 'target': 'target', 'epochs': 50, 'batch_size': 256

In [75]:
from jarvis.db.figshare import data as load_jarvis_data
dataset=load_jarvis_data("dft_3d")
print(f"Loaded {len(dataset)} entries.")

Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
Loaded 55723 entries.


In [76]:
from jarvis.db.figshare import data as load_jarvis_data
import pandas as pd

# Load the dataset
dataset = load_jarvis_data("dft_3d")
print(f"Loaded {len(dataset)} entries.")

# Extract 'jid' and 'formation_energy_peratom'
rows = []
for entry in dataset:
    rows.append({
        'id': entry['jid'],  # 'jid' is the ID
        'formation_energy_peratom': entry['formation_energy_peratom']
    })

# Convert to DataFrame
df = pd.DataFrame(rows)

# Save as CSV
df.to_csv("id_prop.csv", index=False)

print("id_prop.csv saved!")


Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
Loaded 55723 entries.
id_prop.csv saved!


In [77]:
!head id_prop.csv


id,formation_energy_peratom
JVASP-90856,-0.42762
JVASP-86097,-0.41596
JVASP-64906,0.04847
JVASP-98225,-0.4414
JVASP-10,-0.71026
JVASP-14014,-0.45468
JVASP-64664,0.09202
JVASP-22556,-2.07159
JVASP-86726,-0.42539


In [ ]:
from google.colab import files
files.download("id_prop.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [84]:
pwd

'/content/DPF/matformer'

In [85]:
from jarvis.db.figshare import data as load_jarvis_data
from jarvis.core.atoms import Atoms
import os

def atoms_to_poscar(atoms: Atoms) -> str:
    """
    Convert Jarvis Atoms object to POSCAR string format.
    """
    lattice = atoms.lattice_mat
    species = atoms.elements  # list of element symbols like ['Si', 'Si']
    unique_species = sorted(set(species), key=species.index)  # unique preserving order
    counts = [species.count(s) for s in unique_species]
    coords = atoms.cart_coords  # Cartesian coordinates

    lines = []
    lines.append("Generated by script")  # comment line
    lines.append("1.0")  # universal scaling factor

    # lattice vectors (3 lines)
    for v in lattice:
        lines.append(f"{v[0]:.16f} {v[1]:.16f} {v[2]:.16f}")

    # element symbols line
    lines.append(" ".join(unique_species))

    # counts line
    lines.append(" ".join(str(c) for c in counts))

    # coordinate type
    lines.append("Cartesian")

    # atomic positions
    for el in unique_species:
        for i, sp in enumerate(species):
            if sp == el:
                x, y, z = coords[i]
                lines.append(f"{x:.16f} {y:.16f} {z:.16f}")

    return "\n".join(lines)


output_dir = "/content/DPF/poscar_files"
os.makedirs(output_dir, exist_ok=True)

dataset = load_jarvis_data("dft_3d")

for entry in dataset:
    jid = entry['jid']
    atoms_dict = entry['atoms']
    atoms = Atoms.from_dict(atoms_dict)

    poscar_content = atoms_to_poscar(atoms)
    poscar_path = os.path.join(output_dir, f"{jid}.vasp")

    with open(poscar_path, "w") as f:
        f.write(poscar_content)

print(f"Saved POSCAR (.vasp) files to {output_dir}")


Obtaining 3D dataset 55k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Loading the zipfile...
Loading completed.
Saved POSCAR (.vasp) files to /content/DPF/poscar_files


In [87]:
pwd

'/content/DPF/matformer'

In [79]:
%cd /content/DPF/matformer

/content/DPF/matformer


In [100]:
!python pretrain_accelerator_reconstruct.py --root_dir /content/DPF --output_dir /content/DPF/output --batch_size 64 --epochs 10


Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.0+cu118
name='matformer' conv_layers=5 edge_layers=0 atom_input_features=92 edge_features=128 triplet_input_features=40 node_features=128 fc_layers=1 fc_features=128 output_features=1 node_layer_head=4 edge_layer_head=4 nn_based=False link='identity' zero_inflated=False use_angle=False angle_lattice=False classification=False pre_train=False position_noise=None lattice_noise=None mask_ratio=None
{'dataset': 'user_data', 'target': 'target', 'epochs': '10', 'batch_size': '64', 'weight_decay': 1e-05, 'learning_rate': 0.001, 'criterion': 'mse', 'optimizer': 'adamw', 'scheduler': 'onecycle', 'save_dataloader': False, 'pin_memory': False, 'write_predictions': True, 'num_workers': 16, 'classification_threshold': None, 'atom_features': 'atomic_number', 'pre_train': True, 'pyg_input': True, 'use_lattice': True, 'use_angle': False, 'output_dir': '/content/DPF/outputmNone_lNone_pNone/', 'model': {'use_angle': False, 'name': 'matfo

In [98]:
import pandas as pd
s=pd.read_csv("id_prop.csv")
s

,id,formation_energy_peratom
0,JVASP-90856,-0.42762
1,JVASP-86097,-0.41596
2,JVASP-64906,0.04847
3,JVASP-98225,-0.44140
4,JVASP-10,-0.71026
...,...,...
55718,JVASP-123206,0.37366
55719,JVASP-122142,-0.01992
55720,JVASP-123207,0.07529
55721,JVASP-121802,-2.45757
